# **Chapter 10. AI Engineering Architecture and User Feedback**

지금까지 이 책에서는 기본 모델을 특정 응용 프로그램에 맞게 조정하는 다양한 기술을 다루었습니다. 이 장에서는 이러한 기술들을 어떻게 결합하여 성공적인 제품을 구축할 수 있는지를 논의합니다.

사용할 수 있는 AI 엔지니어링 기술과 도구의 범위가 넓기 때문에 올바른 도구를 선택하는 것이 부담스러울 수 있습니다. 이 과정을 단순화하기 위해, 이 장은 점진적인 접근 방식을 취합니다. 가장 단순한 기본 모델 응용 프로그램 아키텍처에서 시작하여, 해당 아키텍처의 문제점을 강조하고, 이를 해결하기 위해 점진적으로 구성 요소를 추가해 나갑니다.

성공적인 응용 프로그램을 구축하는 방법에 대해 끝없이 고민할 수 있지만, 응용 프로그램이 실제로 목표를 달성하는지를 확인하는 유일한 방법은 사용자 앞에 그것을 배치하는 것입니다. 사용자 피드백은 제품 개발을 안내하는 데 항상 귀중한 역할을 해왔지만, AI 응용 프로그램의 경우 사용자 피드백은 모델 개선을 위한 데이터 소스로서 더욱 중요한 역할을 합니다. 대화형 인터페이스는 사용자가 피드백을 제공하기 쉽게 만들지만, 개발자가 그 신호를 추출하기는 더 어렵게 만듭니다. 이 장에서는 다양한 유형의 대화형 AI 피드백과 사용자 경험을 해치지 않으면서 적절한 피드백을 수집하도록 시스템을 설계하는 방법을 논의합니다.

---

## **AI Engineering Architecture**

완전한 형태의 AI 아키텍처는 복잡할 수 있습니다. 이 절에서는 실제 프로덕션 환경에서 팀이 따를 수 있는 프로세스를 따르며, 가장 단순한 아키텍처에서 시작하여 점차적으로 더 많은 구성 요소를 추가해 나갑니다. AI 응용 프로그램의 다양성에도 불구하고, 많은 공통된 구성 요소를 공유합니다. 여기에서 제안하는 아키텍처는 여러 회사에서 다양한 응용 프로그램에 대해 일반적으로 사용될 수 있음이 검증되었지만, 특정 응용 프로그램은 예외일 수 있습니다.

가장 단순한 형태에서, 응용 프로그램은 쿼리를 받아 모델에 전달합니다. 모델은 응답을 생성하고, 그것을 사용자에게 반환합니다. 이는 그림 10-1에 나타나 있습니다. 이 구조에는 컨텍스트 보강도, 가드레일도, 최적화도 없습니다. Model API 박스는 써드파티 API(예: OpenAI, Google, Anthropic)와 자체 호스팅 모델을 모두 포함합니다. 자체 호스팅 모델에 대한 추론 서버 구축은 9장에서 다루었습니다.

<img src="./images/fig_10_01.png" width=800>

Figure 10-1. AI 응용 프로그램을 실행하기 위한 가장 단순한 아키텍처.

이 단순한 아키텍처에서 시작하여 필요에 따라 더 많은 구성 요소를 추가할 수 있습니다. 그 과정은 다음과 같을 수 있습니다:

1. 외부 데이터 소스와 정보 수집 도구에 대한 접근을 모델에 제공하여 컨텍스트 입력을 향상시킵니다.
2. 시스템과 사용자를 보호하기 위한 가드레일을 추가합니다.
3. 복잡한 파이프라인을 지원하고 보안을 강화하기 위해 모델 라우터와 게이트웨이를 추가합니다.
4. 캐싱을 통해 지연 시간과 비용을 최적화합니다.
5. 복잡한 로직과 액션 작성을 추가하여 시스템의 역량을 극대화합니다.

이 장에서는 실제 프로덕션에서 흔히 보는 점진적 발전 과정을 따릅니다. 하지만 각자의 필요는 다릅니다. 응용 프로그램에 가장 적합한 순서를 따라야 합니다.

품질 관리와 성능 향상을 위한 애플리케이션의 필수 요소인 모니터링과 가시성은 이 과정의 마지막에서 논의됩니다. 이러한 모든 구성 요소를 연결하는 오케스트레이션은 그 이후에 다룰 것입니다.

---


### Step 1. Enhance Context

플랫폼의 초기 확장은 일반적으로 시스템이 각 쿼리에 대해 모델이 응답하는 데 필요한 관련 컨텍스트를 구성할 수 있도록 하는 메커니즘을 추가하는 것과 관련됩니다. 6장에서 논의한 바와 같이, 컨텍스트는 텍스트 검색, 이미지 검색, 표 형식 데이터 검색 등 다양한 검색 메커니즘을 통해 구성될 수 있습니다. 컨텍스트는 웹 검색, 뉴스, 날씨, 이벤트 등과 같은 API를 통해 모델이 자동으로 정보를 수집할 수 있도록 하는 도구를 사용하여 확장될 수도 있습니다.

컨텍스트 구성은 기본 모델을 위한 피처 엔지니어링과 같습니다. 이는 모델이 출력을 생성하는 데 필요한 정보를 제공합니다. 시스템 출력 품질에서 중심적인 역할을 하기 때문에, 컨텍스트 구성은 거의 모든 모델 API 제공자에 의해 지원됩니다. 예를 들어, OpenAI, Claude, Gemini와 같은 제공자는 사용자가 파일을 업로드하고 모델이 도구를 사용할 수 있도록 허용합니다.

하지만 모델의 기능이 서로 다른 것처럼, 이러한 제공자들도 컨텍스트 구성 지원에 차이를 보입니다. 예를 들어, 업로드할 수 있는 문서의 유형과 수량에 제한이 있을 수 있습니다. 특화된 RAG 솔루션은 벡터 데이터베이스가 수용할 수 있는 만큼 문서를 업로드할 수 있도록 하지만, 일반적인 모델 API는 소량의 문서만 업로드하도록 제한할 수 있습니다. 다양한 프레임워크는 검색 알고리즘 및 청크 크기와 같은 검색 구성에서도 차이를 보입니다. 도구 사용의 경우에도, 솔루션은 지원하는 도구의 유형과 실행 모드(예: 병렬 함수 실행 또는 장기 실행 작업 지원 여부) 측면에서 차이를 보입니다.

컨텍스트 구성이 추가되면, 아키텍처는 이제 그림 10-2와 같은 형태가 됩니다.

<img src="./images/fig_10_02.png" width=800>

Figure 10-2. 컨텍스트 구성이 포함된 플랫폼 아키텍처.

---


### Step 2. Put in Guardrails

가드레일은 위험을 완화하고 당신과 사용자를 보호하는 데 도움을 줍니다. 위험에 노출되는 부분이 있다면 언제든지 가드레일을 두어야 합니다. 일반적으로 가드레일은 입력과 출력에 대한 것으로 분류할 수 있습니다.

**Input guardrails**

입력 가드레일은 일반적으로 두 가지 유형의 위험으로부터 보호합니다: 외부 API에 민감 정보를 유출하는 것과 시스템을 손상시키는 악성 프롬프트 실행입니다. 5장에서 공격자가 프롬프트 해킹을 통해 애플리케이션을 악용하는 다양한 방식과 이를 방어하는 방법을 논의했습니다. 위험을 완화할 수는 있지만, 모델이 응답을 생성하는 본질적인 방식과 피할 수 없는 인간의 실수 때문에 위험을 완전히 제거할 수는 없습니다.

외부 모델 API를 사용할 때 데이터를 조직 외부로 전송해야 하므로, 외부 API에 민감 정보를 유출하는 위험이 발생할 수 있습니다. 이러한 일은 다음과 같은 다양한 이유로 발생할 수 있습니다:

* 직원이 회사의 비밀이나 사용자의 개인 정보를 프롬프트에 복사하여 서드파티 API에 전송하는 경우 ${ }^{1}$
* 애플리케이션 개발자가 회사의 내부 정책이나 데이터를 애플리케이션의 시스템 프롬프트에 삽입한 경우
* 도구가 내부 데이터베이스에서 민감한 정보를 검색하여 컨텍스트에 추가한 경우

서드파티 API를 사용할 때 잠재적 유출을 완벽히 차단하는 방법은 없습니다. 그러나 가드레일을 통해 이를 완화할 수는 있습니다. 민감 정보를 자동으로 탐지하는 다양한 도구 중 하나를 사용할 수 있습니다. 어떤 민감 정보를 탐지할지는 사용자가 직접 지정합니다. 일반적인 민감 정보 클래스는 다음과 같습니다:

* 개인 정보(주민등록번호, 전화번호, 은행 계좌 등)
* 사람 얼굴
* 회사의 지식재산 또는 기밀 정보와 관련된 특정 키워드 및 문구

많은 민감 정보 탐지 도구는 AI를 사용하여 민감할 가능성이 있는 정보를 식별합니다. 예를 들어, 문자열이 유효한 자택 주소처럼 보이는지 판단합니다. 쿼리에서 민감 정보가 발견되면, 두 가지 선택지가 있습니다: 전체 쿼리를 차단하거나, 민감 정보를 제거하는 것입니다. 예를 들어, 사용자의 전화번호를 \[PHONE NUMBER] 자리 표시자로 마스킹할 수 있습니다. 생성된 응답에 이 자리 표시자가 포함되어 있다면,

그 자리 표시자를 원래 정보로 다시 바꾸기 위해 PII 역사전을 사용할 수 있습니다. 그림 10-3에 그 예시가 나와 있습니다.

<img src="./images/fig_10_03.png" width=800>

Figure 10-3. 외부 API로 전송하지 않기 위해 역 PII 맵을 사용하여 PII 정보를 마스킹하고 복원하는 예시.

**Output guardrails**

모델은 다양한 방식으로 실패할 수 있습니다. 출력 가드레일은 두 가지 주요 기능을 수행합니다:

* 출력 실패 탐지
* 다양한 실패 유형을 처리하는 정책 지정

모델이 기준에 미달하는 출력을 생성할 때 이를 탐지하려면, 실패가 어떤 모습인지 이해해야 합니다. 가장 탐지하기 쉬운 실패는 모델이 출력해서는 안 될 때 빈 응답을 반환하는 경우입니다 ${ }^{2}$. 실패는 애플리케이션마다 다르게 나타납니다. 여기 두 가지 주요 범주인 품질과 보안에서 자주 발생하는 실패 사례가 있습니다. 품질 실패는 4장에서, 보안 실패는 5장에서 다루었습니다. 다음은 그 요약입니다:

* 품질

  * 예상한 출력 형식을 따르지 않는 잘못된 형식의 응답. 예를 들어, 애플리케이션이 JSON을 기대하는데 모델이 유효하지 않은 JSON을 생성하는 경우.
  * 모델이 환각으로 생성한 사실과 일치하지 않는 응답.
  * 일반적으로 품질이 낮은 응답. 예를 들어, 모델에 에세이를 작성하도록 했는데 그 에세이가 형편없는 경우.
* 보안

  * 인종차별적 내용, 성적인 내용, 불법 행위를 포함한 유해한 응답.
  * 개인 정보 및 민감한 정보를 포함한 응답.
  * 원격 도구 실행 또는 코드 실행을 유발하는 응답.
  * 회사 또는 경쟁사에 대한 왜곡된 설명을 포함한 브랜드 위험 응답.

5장에서 언급했듯이, 보안 측정을 할 때는 보안 실패뿐만 아니라 잘못된 차단률(false refusal rate)도 추적하는 것이 중요합니다. 너무 보안이 강한 시스템은 합법적인 요청까지 차단하여 사용자 작업을 방해하고 불만을 초래할 수 있습니다.

많은 실패는 단순한 재시도 로직으로 완화할 수 있습니다. AI 모델은 확률적이기 때문에, 동일한 쿼리를 다시 시도하면 다른 응답이 나올 수 있습니다. 예를 들어, 응답이 비어 있다면 X번 재시도하거나 비어 있지 않은 응답이 나올 때까지 시도할 수 있습니다. 마찬가지로, 형식이 잘못된 응답이라면, 올바른 형식이 나올 때까지 다시 시도할 수 있습니다.

하지만 이 재시도 정책은 지연 시간과 비용이 추가로 발생할 수 있습니다. 재시도마다 API 호출이 추가로 발생하기 때문입니다. 실패 후 재시도를 수행하면 사용자 체감 지연 시간이 두 배로 늘어날 수 있습니다. 지연 시간을 줄이기 위해 병렬로 호출할 수 있습니다. 예를 들어, 각 쿼리에 대해 첫 번째 쿼리 실패를 기다리지 않고, 모델에 두 번 동시에 쿼리를 보내고 두 개의 응답 중 더 나은 것을 선택합니다. 이렇게 하면 중복된 API 호출 수는 증가하지만 지연 시간은 관리 가능합니다.

복잡한 요청의 경우 사람에게 전달하는 것도 일반적입니다. 예를 들어, 특정 문구를 포함하는 쿼리를 인간 상담사에게 전달할 수 있습니다. 어떤 팀은 대화를 언제 사람에게 전달할지를 결정하기 위해 특수한 모델을 사용합니다. 한 팀은 감성 분석 모델이 사용자 메시지에서 분노를 감지하면 대화를 사람에게 전달합니다. 또 다른 팀은 사용자가 루프에 갇히는 것을 방지하기 위해 일정 횟수의 대화 이후에는 사람에게 넘깁니다.

**Guardrail implementation**

가드레일에는 트레이드오프가 따릅니다. 그 중 하나는 신뢰성과 지연 시간 사이의 트레이드오프입니다. 가드레일의 중요성을 인정하면서도, 어떤 팀은 지연 시간이 더 중요하다고 말했습니다. 이 팀들은 가드레일이 애플리케이션의 지연 시간을 크게 증가시킬 수 있기 때문에 가드레일을 구현하지 않기로 결정했습니다. ${ }^{3}$

출력 가드레일은 스트림 완료 모드에서는 제대로 작동하지 않을 수 있습니다. 기본적으로 전체 응답이 생성된 후 사용자에게 표시되며, 이 과정은 시간이 오래 걸릴 수 있습니다. 스트림 완료 모드에서는 새 토큰이 생성되는 즉시 사용자에게 스트리밍되므로, 사용자가 응답을 보기까지 기다리는 시간이 줄어듭니다. 단점은 부분 응답을 평가하기 어렵다는 점이며, 이로 인해 시스템 가드레일이 차단해야 한다고 판단하기 전에 비안전한 응답이 사용자에게 스트리밍될 수 있습니다.

얼마나 많은 가드레일을 구현해야 하는지는 모델을 자체 호스팅하는지 또는 서드파티 API를 사용하는지에 따라 달라집니다. 두 경우 모두 위에 가드레일을 구현할 수 있지만, 서드파티 API는 일반적으로 기본적으로 많은 가드레일을 제공하므로 구현해야 할 가드레일의 수를 줄일 수 있습니다. 동시에, 자체 호스팅은 외부로 요청을 보낼 필요가 없으므로 많은 유형의 입력 가드레일 필요성을 줄여줍니다.

애플리케이션이 실패할 수 있는 지점이 많기 때문에, 다양한 수준에서 가드레일을 구현할 수 있습니다. 모델 제공자들은 모델을 더 좋고 안전하게 만들기 위해 자체 모델에 가드레일을 제공합니다. 그러나 모델 제공자는 안전성과 유연성의 균형을 맞춰야 합니다. 제한이 모델을 더 안전하게 만들 수 있지만, 특정 사용 사례에서는 모델의 사용성을 떨어뜨릴 수 있습니다.

가드레일은 애플리케이션 개발자에 의해서도 구현될 수 있습니다. "프롬프트 공격에 대한 방어"에서 많은 기술이 논의되었습니다. 즉시 사용할 수 있는 가드레일 솔루션에는 Meta의 Purple Llama, NVIDIA의 NeMo Guardrails, Azure의 PyRIT, Azure의 AI 콘텐츠 필터, Perspective API,

그리고 OpenAI의 콘텐츠 모더레이션 API가 있습니다. 입력과 출력의 위험이 겹치는 경우가 많기 때문에, 대부분의 가드레일 솔루션은 입력과 출력을 모두 보호합니다. 일부 모델 게이트웨이도 다음 섹션에서 논의할 가드레일 기능을 제공합니다.

가드레일이 추가되면 아키텍처는 그림 10-4와 같은 형태가 됩니다. 나는 모델 API 아래에 스코어러를 배치했습니다. 왜냐하면 스코어러는 일반적으로 생성 모델보다 작고 빠르지만, 종종 AI 기반이기 때문입니다. 그러나 스코어러는 출력 가드레일 박스에 배치할 수도 있습니다.

<img src="./images/fig_10_04.png" width=800>

Figure 10-4. 입력 및 출력 가드레일이 추가된 애플리케이션 아키텍처.

---


### Step 3. Add Model Router and Gateway

애플리케이션이 여러 모델을 포함하게 되면서, 라우터와 게이트웨이는 여러 모델을 서비스하는 데 따르는 복잡성과 비용을 관리하는 데 도움이 됩니다.

**Router**

모든 쿼리에 하나의 모델을 사용하는 대신, 쿼리 유형에 따라 서로 다른 솔루션을 사용할 수 있습니다. 이 접근 방식은 여러 가지 이점을 제공합니다. 첫째, 특정 쿼리에 대해 범용 모델보다 성능이 더 좋은 특화 모델을 사용할 수 있습니다. 예를 들어, 기술 지원에 특화된 모델 하나와 결제 관련 요청에 특화된 또 다른 모델을 사용할 수 있습니다. 둘째, 비용 절감에 도움이 될 수 있습니다. 모든 쿼리에 비싼 모델을 사용하는 대신, 더 간단한 쿼리는 저렴한 모델로 라우팅할 수 있습니다.

라우터는 일반적으로 사용자의 의도를 예측하는 인텐트 분류기로 구성됩니다. 예측된 의도에 따라 쿼리는 적절한 솔루션으로 라우팅됩니다. 예를 들어, 고객 지원 챗봇과 관련된 다양한 의도를 고려해보면 다음과 같습니다:

* 사용자가 비밀번호 재설정을 원할 경우, 비밀번호 복구에 관한 FAQ 페이지로 라우팅합니다.
* 결제 오류를 수정하려는 요청일 경우, 인간 운영자에게 라우팅합니다.
* 기술 문제 해결에 관한 요청일 경우, 문제 해결에 특화된 챗봇에 라우팅합니다.

인텐트 분류기는 시스템이 범위를 벗어난 대화에 빠지는 것을 방지할 수 있습니다. 쿼리가 부적절하다고 판단되면 챗봇은 API 호출을 낭비하지 않고 정해진 응답 중 하나를 사용해 정중히 응답을 거절할 수 있습니다. 예를 들어, 사용자가 다가오는 선거에서 누구를 지지하느냐고 묻는 경우, 챗봇은 다음과 같이 응답할 수 있습니다: "저는 챗봇이기 때문에 투표권이 없습니다. 저희 제품에 대해 궁금하신 점이 있다면 기꺼이 도와드리겠습니다."

인텐트 분류기는 시스템이 모호한 쿼리를 감지하고 명확히 하도록 도울 수 있습니다. 예를 들어, 사용자가 "Freezing"이라고 입력하면, 시스템은 "계정을 동결하고 싶으신가요, 아니면 날씨에 대해 말씀하시는 건가요?"라고 묻거나, 간단히 "죄송합니다. 좀 더 자세히 설명해 주시겠어요?"라고 물을 수 있습니다.

다른 라우터는 모델이 다음에 무엇을 해야 할지를 결정하는 데 도움을 줄 수 있습니다. 예를 들어, 여러 작업을 수행할 수 있는 에이전트의 경우, 라우터는 다음 동작 예측기(next-action predictor)의 형태를 가질 수 있습니다: 모델이 다음에 코드 인터프리터를 사용해야 할지, 검색 API를 사용해야 할지를 결정하는 것입니다. 메모리 시스템이 있는 모델의 경우, 라우터는 모델이 메모리 계층 중 어디에서 정보를 가져와야 하는지를 예측할 수 있습니다. 사용자가 현재 대화에 멜버른을 언급한 문서를 첨부했다고 가정해 봅시다. 이후에 사용자가 "멜버른에서 가장 귀여운 동물은 뭐야?"라고 묻는다면, 모델은 이 쿼리를 해결하기 위해 첨부된 문서의 정보를 사용할지, 인터넷에서 검색할지를 결정해야 합니다.

인텐트 분류기와 다음 동작 예측기는 기본 모델 위에 구현될 수 있습니다. 많은 팀들이 GPT-2, BERT, Llama 7B와 같은 작은 언어 모델을 인텐트 분류기로 적응시킵니다. 많은 팀들이 훨씬 작은 분류기를 처음부터 직접 학습시키기도 합니다. 라우터는 빠르고 저렴해야 하므로 여러 개를 사용해도 큰 지연이나 비용이 발생하지 않아야 합니다.

컨텍스트 제한이 다양한 모델로 쿼리를 라우팅할 때는 쿼리의 컨텍스트를 적절히 조정해야 할 수도 있습니다. 예를 들어, 1,000 토큰짜리 쿼리를 컨텍스트 제한이 4K인 모델에 보낼 예정인데, 시스템이 웹 검색과 같은 작업을 통해 8,000 토큰의 컨텍스트를 되돌려주는 경우를 생각해보세요. 이때 쿼리의 컨텍스트를 잘라 원래 모델에 맞추거나, 더 큰 컨텍스트 제한을 가진 모델로 라우팅할 수 있습니다.

라우팅은 일반적으로 모델에 의해 수행되므로, 그림 10-5에서는 라우팅을 Model API 박스 내부에 배치했습니다. 스코어러와 마찬가지로, 라우터는 일반적으로 생성 모델보다 작습니다.

라우터를 다른 모델들과 함께 그룹화하면 모델 관리를 더 쉽게 할 수 있습니다. 하지만 라우팅은 종종 검색 전에 이루어진다는 점에 유의해야 합니다. 예를 들어, 검색 전에 라우터는 쿼리가 범위 내인지 판단하고, 필요한 경우에만 검색을 수행할 수 있습니다. 라우팅은 검색 후에 이루어질 수도 있으며, 예를 들어 쿼리를 인간 운영자에게 라우팅할지를 결정하는 경우입니다. 그러나 라우팅 - 검색 - 생성 - 스코어링은 훨씬 더 일반적인 AI 애플리케이션 패턴입니다.

<img src="./images/fig_10_05.png" width=800>

Figure 10-5. 라우팅은 시스템이 각 쿼리에 대해 최적의 솔루션을 사용하도록 돕습니다.

**Gateway**

모델 게이트웨이는 다양한 모델과 통합하고 안전하게 인터페이스할 수 있도록 해주는 중간 계층입니다. 모델 게이트웨이의 가장 기본적인 기능은 자체 호스팅 모델과 상용 API 모델을 포함하여 다양한 모델에 대해 통합된 인터페이스를 제공하는 것입니다. 모델 게이트웨이는 코드 유지보수를 용이하게 만듭니다. 모델 API가 변경되더라도, 해당 API에 의존하는 모든 애플리케이션을 수정하는 대신 게이트웨이만 업데이트하면 됩니다. 그림 10-6은 모델 게이트웨이의 상위 수준 시각화를 보여줍니다.

<img src="./images/fig_10_06.png" width=800>

Figure 10-6. 모델 게이트웨이는 다양한 모델과 함께 작동하기 위한 통합 인터페이스를 제공합니다.

가장 단순한 형태에서, 모델 게이트웨이는 통합 래퍼입니다. 다음 코드는 모델 게이트웨이가 어떻게 구현될 수 있는지를 보여주는 예시입니다. 이 코드는 오류 확인이나 최적화가 포함되어 있지 않기 때문에 실행 가능한 것은 아닙니다:

```python
import google.generativeai as genai
import openai

def openai_model(input_data, model_name, max_tok(
    openai.api_key = os.environ["OPENAI_API_KEY"
    response = openai.Completion.create(
        engine=model_name,
        prompt=input_data,
        max_tokens=max_tokens
    )
    return {"response": response.choices[0].text

def gemini_model(input_data, model_name, max_tok(
    genai.configure(api_key=os.environ["GOOGLE_AI
    model = genai.GenerativeModel(model_name=modi
    response = model.generate_content(input_data
    return {"response": response["choices"][0]["1

@app.route('/model', methods=['POST'])
def model_gateway():
    data = request.get_json()
    model_type = data.get("model_type")
    model_name = data.get("model_name")
    input_data = data.get("input_data")
    max_tokens = data.get("max_tokens")
    if model_type == "openai":
        result = openai_model(input_data, 1
    elif model_type == "gemini":
        result = gemini_model(input_data, 1
    return jsonify(result)
```

모델 게이트웨이는 접근 제어와 비용 관리를 제공합니다. OpenAI API에 접근하려는 모든 사람에게 조직의 토큰을 제공하는 대신, 모델 게이트웨이에만 접근 권한을 주면 되므로 유출 위험이 줄어듭니다. 게이트웨이는 어떤 사용자 또는 애플리케이션이 어떤 모델에 접근할 수 있는지를 지정하는 세분화된 접근 제어도 구현할 수 있습니다. 또한, 게이트웨이는 API 호출 사용량을 모니터링하고 제한함으로써 오남용을 방지하고 비용을 효과적으로 관리할 수 있습니다.

모델 게이트웨이는 속도 제한이나 API 실패(이는 안타깝게도 흔합니다)를 극복하기 위한 폴백(fallback) 정책을 구현하는 데에도 사용할 수 있습니다. 기본 API가 사용할 수 없을 때, 게이트웨이는 요청을 대체 모델로 라우팅하거나, 잠시 기다린 후 재시도하거나, 그 외 다른 방식으로 실패를 우아하게 처리할 수 있습니다. 이는 애플리케이션이 중단 없이 원활히 작동할 수 있도록 보장합니다.

요청과 응답이 이미 게이트웨이를 통과하므로, 로드 밸런싱, 로깅, 분석 등의 기능을 구현하기에 적절한 위치입니다. 일부 게이트웨이는 캐싱 및 가드레일 기능까지 제공합니다.

게이트웨이는 비교적 구현이 간단하기 때문에, 많은 기성 게이트웨이가 존재합니다. 예로는 Portkey의 AI Gateway, MLflow AI Gateway, Wealthsimple의 LLM Gateway, TrueFoundry, Kong, Cloudflare 등이 있습니다.

우리의 아키텍처에서는 이제 게이트웨이가 모델 API 박스를 대체하며, 그림 10-7에 나타나 있습니다.

<img src="./images/fig_10_07.png" width=800>

Figure 10-7. 라우팅과 게이트웨이 모듈이 추가된 아키텍처.

> **NOTE**
>
> 다양한 도구에 접근하기 위한 유사한 추상화 계층인 툴 게이트웨이도 유용할 수 있습니다. 하지만 이 책에서는 현재 시점에서 일반적인 패턴이 아니기 때문에 다루지 않습니다.

---

### Step 4. Reduce Latency with Caches

캐싱은 지연 시간과 비용을 줄이기 위해 소프트웨어 애플리케이션에서 오랫동안 핵심적인 역할을 해왔습니다. 소프트웨어 캐싱에서의 많은 아이디어는 AI 애플리케이션에도 적용될 수 있습니다. KV 캐싱과 프롬프트 캐싱을 포함한 추론 캐싱 기법은 9장에서 논의되었습니다. 이 절에서는 시스템 캐싱에 초점을 맞춥니다.

캐싱은 오래된 기술로서 방대한 기존 문헌이 존재하기 때문에, 이 책에서는 이를 넓은 개요 수준에서 다룹니다. 일반적으로 시스템 캐싱 메커니즘에는 정확 캐싱과 의미 캐싱의 두 가지 주요 방식이 있습니다.

**Exact caching**

정확 캐싱에서는 요청된 항목이 이전에 정확히 동일하게 캐싱된 경우에만 캐시된 항목이 사용됩니다. 예를 들어, 사용자가 제품 요약을 요청하면, 시스템은 해당 제품에 대한 요약이 캐시에 존재하는지 확인합니다. 존재한다면 해당 요약을 가져오고, 존재하지 않으면 제품을 요약한 후 결과를 캐싱합니다.

정확 캐싱은 중복 벡터 검색을 피하기 위해 임베딩 기반 검색에도 사용됩니다. 들어오는 쿼리가 이미 벡터 검색 캐시에 있다면, 캐시된 결과를 가져오고, 그렇지 않으면 해당 쿼리에 대해 벡터 검색을 수행하고 그 결과를 캐싱합니다.

캐싱은 여러 단계(예: chain-of-thought)를 포함하거나 시간이 많이 걸리는 작업(예: 검색, SQL 실행, 웹 검색)이 포함된 쿼리에 특히 매력적입니다.

정확 캐시는 빠른 검색을 위해 인메모리 저장소를 사용하여 구현할 수 있습니다. 그러나 인메모리 저장소는 제한되어 있으므로, PostgreSQL, Redis와 같은 데이터베이스나 계층형 저장소를 사용하여 속도와 저장 용량을 균형 있게 조절할 수 있습니다. 캐시 크기를 관리하고 성능을 유지하기 위해서는 제거 정책(eviction policy)이 필수적입니다. 일반적인 제거 정책에는 LRU(가장 최근에 사용되지 않은 항목 제거), LFU(가장 적게 사용된 항목 제거), FIFO(먼저 들어온 항목부터 제거)가 있습니다.

쿼리를 캐시에 얼마나 오래 유지할지는 해당 쿼리가 다시 호출될 가능성에 따라 달라집니다. 예를 들어, "내 최근 주문 상태는?"과 같은 사용자 특정 쿼리는 다른 사용자가 재사용할 가능성이 낮으므로 캐싱하지 않는 것이 좋습니다. 마찬가지로, "지금 날씨 어때?"와 같은 시간에 민감한 쿼리를 캐싱하는 것도 적절하지 않습니다. 많은 팀들은 쿼리를 캐싱할지 예측하기 위한 분류기를 학습시킵니다.

>**WARNING**
>
>캐싱을 적절히 처리하지 않으면 데이터 유출이 발생할 수 있습니다. 예를 들어, 당신이 이커머스 사이트에서 일한다고 가정합시다. 사용자 X가 "전자제품의 반품 정책은 무엇인가요?"와 같은 일반적인 질문을 합니다. 하지만 반품 정책은 사용자 멤버십에 따라 달라지기 때문에, 시스템은 먼저 사용자 X의 정보를 조회하고 X의 정보를 포함한 응답을 생성합니다. 이 쿼리를 일반 질문으로 오인한 시스템이 응답을 캐시에 저장하면, 이후 사용자 Y가 동일한 질문을 했을 때 캐시된 응답이 반환되어 X의 정보가 Y에게 노출될 수 있습니다.

**Semantic caching**

정확 캐싱과 달리, 의미 캐싱에서는 쿼리가 정확히 일치하지 않더라도 의미적으로 유사하다면 캐시된 항목이 사용됩니다.

예를 들어, 한 사용자가 "베트남의 수도는 어디인가요?"라고 질문하고 모델이 "하노이"라고 응답했다고 가정합시다. 이후 다른 사용자가 "베트남의 수도 도시는 무엇인가요?"라고 묻는다면, 이는 표현만 다를 뿐 의미는 동일한 질문입니다. 의미 캐싱을 사용하면, 시스템은 처음 쿼리의 응답을 재사용할 수 있습니다. 이처럼 유사한 쿼리를 재사용하면 캐시 적중률이 증가하고 비용을 줄일 수 있습니다. 그러나 의미 캐싱은 모델 성능을 저하시킬 수 있습니다.

의미 캐싱은 두 쿼리가 유사한지를 신뢰성 있게 판단할 수 있을 때만 작동합니다. 일반적인 접근 방식은 3장에서 논의된 의미 유사도를 활용하는 것입니다. 간단히 요약하면, 의미 유사도는 다음과 같이 작동합니다:

1. 각 쿼리에 대해 임베딩 모델을 사용하여 임베딩을 생성합니다.
2. 벡터 검색을 통해 현재 쿼리 임베딩과 가장 높은 유사도를 가지는 캐시된 임베딩을 찾습니다. 이 유사도 점수를 X라고 합시다.
3. X가 특정 유사도 임계값보다 높으면, 해당 캐시된 쿼리는 유사한 것으로 간주되어 캐시된 결과를 반환합니다. 그렇지 않으면 현재 쿼리를 처리하고 그 임베딩과 결과를 함께 캐싱합니다.

이 접근 방식에는 캐시된 쿼리의 임베딩을 저장하기 위한 벡터 데이터베이스가 필요합니다.

다른 캐싱 기법과 비교했을 때, 의미 캐싱의 가치는 다소 의심스러울 수 있습니다. 왜냐하면 그 구성 요소들이 실패하기 쉽기 때문입니다. 이 방식의 성공은 고품질 임베딩, 제대로 작동하는 벡터 검색, 신뢰할 수 있는 유사도 지표에 달려 있습니다. 적절한 유사도 임계값을 설정하는 것도 어렵고, 많은 시행착오가 필요할 수 있습니다. 시스템이 들어온 쿼리를 기존 쿼리와 유사하다고 잘못 판단하면, 잘못된 캐시 응답을 반환하게 됩니다.

게다가, 의미 캐싱은 벡터 검색이 포함되므로 시간과 연산 비용이 많이 듭니다. 이 벡터 검색의 속도와 비용은 캐시된 임베딩의 크기에 따라 달라집니다.

그러나 캐시 적중률이 높다면, 의미 캐싱도 여전히 가치 있을 수 있습니다. 즉, 많은 쿼리가 캐시된 결과로 효과적으로 응답될 수 있다면 유용할 수 있습니다. 하지만 의미 캐싱의 복잡성을 도입하기 전에 관련된 효율성, 비용, 성능 위험을 반드시 평가해야 합니다.

캐시 시스템이 추가되면 플랫폼은 그림 10-8과 같은 형태가 됩니다. KV 캐시와 프롬프트 캐시는 일반적으로 모델 API 제공자에 의해 구현되므로 이 이미지에는 나타나지 않습니다. 그것들을 시각화하려면 Model API 박스에 넣으면 됩니다. 생성된 응답을 캐시에 추가하는 새로운 화살표가 있습니다.

<img src="./images/fig_10_08.png" width=800>

Figure 10-8. 캐시가 추가된 AI 애플리케이션 아키텍처.

---

### Step 5. Add Agent Patterns

지금까지 논의된 애플리케이션은 여전히 비교적 단순한 구조입니다. 각 쿼리는 순차적인 흐름을 따릅니다. 그러나 6장에서 논의한 바와 같이, 애플리케이션 흐름은 루프, 병렬 실행, 조건 분기 등으로 더 복잡해질 수 있습니다. 6장에서 다룬 에이전트 패턴(agentic patterns)은 복잡한 애플리케이션을 구축하는 데 도움이 됩니다. 예를 들어, 시스템이 출력을 생성한 후 작업이 완료되지 않았다고 판단하고, 추가적인 정보를 수집하기 위해 다시 검색을 수행할 수 있습니다. 원래의 응답은 새로 검색된 컨텍스트와 함께 동일한 모델 또는 다른 모델에 다시 전달됩니다. 이는 그림 10-9에 표시된 것처럼 루프를 형성합니다.

<img src="./images/fig_10_09.png" width=800>

Figure 10-9. 생성된 응답이 시스템에 다시 입력되어 더 복잡한 애플리케이션 패턴을 가능하게 하는 노란색 화살표.

모델의 출력은 이메일 작성, 주문 처리, 은행 송금 시작과 같은 쓰기 작업(write actions)을 트리거하는 데에도 사용될 수 있습니다. 쓰기 작업은 시스템이 직접 환경에 변화를 줄 수 있도록 합니다. 6장에서 논의한 바와 같이, 쓰기 작업은 시스템의 역량을 크게 향상시킬 수 있지만 동시에 훨씬 더 많은 위험에도 노출시킵니다. 모델에 쓰기 작업에 대한 접근 권한을 부여할 때는 극도로 신중해야 합니다. 쓰기 작업이 추가되면 아키텍처는 그림 10-10과 같은 형태가 됩니다.

지금까지의 모든 단계를 따라왔다면, 아키텍처는 상당히 복잡해졌을 것입니다. 복잡한 시스템은 더 많은 작업을 수행할 수 있지만, 실패 지점이 많아 디버깅이 어려워지고 실패 모드가 늘어납니다. 다음 섹션에서는 시스템의 가시성을 향상시키기 위한 모범 사례를 다룰 것입니다.

<img src="./images/fig_10_10.png" width=800>

Figure 10-10. 시스템이 쓰기 작업을 수행할 수 있도록 하는 애플리케이션 아키텍처.

**Monitoring and Observability**

비록 가시성을 별도의 섹션으로 다루고 있지만, 가시성은 제품 설계에 있어 후속 고려사항이 아니라 필수 요소로 포함되어야 합니다. 제품이 복잡해질수록 가시성의 중요성도 커집니다.

가시성은 모든 소프트웨어 엔지니어링 분야에서 보편적으로 적용되는 관행입니다. 이미 잘 확립된 모범 사례와 많은 상용 및 오픈 소스 솔루션이 존재하는 거대한 산업입니다.$^{4}$ 쓸데없이 바퀴를 다시 발명하지 않기 위해, 이 책에서는 기본 모델 위에 구축된 애플리케이션에 특화된 내용을 중심으로 설명합니다. 책의 GitHub 저장소에는 가시성에 대해 더 깊이 배우고 싶은 사람들을 위한 자료가 포함되어 있습니다. ${ }^{5}$

모니터링의 목표는 평가의 목표와 동일합니다: 위험을 완화하고 기회를 발견하는 것입니다. 모니터링이 완화해야 할 위험에는 애플리케이션 실패, 보안 공격, 드리프트 등이 포함됩니다. 모니터링은 애플리케이션 개선과 비용 절감 기회를 발견하는 데에도 도움을 줄 수 있습니다. 또한, 모니터링은 시스템 성능에 대한 가시성을 제공함으로써 책임성을 유지하는 데에도 기여할 수 있습니다.

DevOps 커뮤니티에서 파생된 세 가지 지표는 시스템의 가시성 품질을 평가하는 데 도움을 줄 수 있습니다:

* MTTD(mean time to detection): 문제가 발생했을 때, 이를 탐지하는 데 걸리는 평균 시간.
* MTTR(mean time to response): 문제를 탐지한 이후 해결까지 걸리는 평균 시간.
* CFR(change failure rate): 수정을 필요로 하거나 롤백을 유발하는 실패로 이어진 변경 또는 배포의 비율. (예시: 10번 배포했는데 3번은 오류가 나서 롤백했으면 CFR은 30%)

CFR을 모른다면, 플랫폼을 더 관찰 가능한 형태로 재설계해야 할 시점입니다. CFR이 높다고 해서 반드시 모니터링 시스템이 나쁘다는 것을 의미하지는 않습니다. 하지만 나쁜 변경 사항이 배포되기 전에 탐지되도록 평가 파이프라인을 재검토해야 합니다. 평가와 모니터링은 밀접하게 협력해야 하며, 평가 지표는 모니터링 지표로 잘 전환되어야 합니다. 즉, 평가에서 좋은 성능을 보인 모델은 모니터링 중에도 좋은 성능을 보여야 하며, 모니터링 중에 탐지된 문제는 평가 파이프라인에 다시 반영되어야 합니다.

---

### Monitoring and Observability

2010년대 중반 이후, 업계는 "모니터링" 대신 "가시성(observability)"이라는 용어를 채택했습니다. 모니터링은 시스템의 내부 상태와 출력 간의 관계에 대한 가정을 하지 않습니다. 시스템 외부 출력을 모니터링하여 내부에서 무언가 잘못되었을 때 이를 파악하려 하지만, 외부 출력만으로는 무엇이 잘못되었는지를 알아낼 수 있다는 보장이 없습니다.

반면, 가시성은 전통적인 모니터링보다 더 강한 가정을 전제로 합니다. 즉, 시스템의 외부 출력을 통해 내부 상태를 추론할 수 있다는 것입니다. 가시성이 있는 시스템에서 문제가 발생하면, 새 코드를 배포하지 않고도 시스템의 로그와 메트릭을 통해 무엇이 잘못되었는지를 파악할 수 있어야 합니다. 가시성이란 시스템의 런타임에 대한 충분한 정보를 수집하고 분석하여, 문제가 발생했을 때 그것이 무엇인지 파악할 수 있도록 시스템을 계측하는 방법입니다.

이 책에서는 "monitoring"을 시스템 정보를 추적하는 행위로, "observability"를 시스템을 계측하고, 추적하고, 디버깅하는 전체 과정으로 정의합니다.

**Metrics**

모니터링에 대해 이야기할 때 대부분의 사람들은 메트릭을 떠올립니다. 하지만 메트릭 자체가 목적은 아닙니다. 솔직히 대부분의 회사는 당신의 애플리케이션 출력 적합성 점수가 얼마인지 자체에는 관심이 없습니다. 메트릭의 목적은 무언가 잘못되었을 때 그것을 알려주고, 개선의 기회를 식별하는 데 있습니다.

어떤 메트릭을 추적할지를 나열하기 전에, 잡고자 하는 실패 모드를 이해하고 해당 실패를 중심으로 메트릭을 설계하는 것이 중요합니다. 예를 들어, 애플리케이션이 환각(hallucination)을 하지 않게 하려면, 환각을 탐지하는 데 도움이 되는 메트릭을 설계해야 합니다. 관련된 메트릭 중 하나는 애플리케이션 출력이 컨텍스트로부터 추론 가능한지 여부일 수 있습니다. API 크레딧을 소모하지 않게 하려면, 요청당 입력 및 출력 토큰 수, 캐시 비용, 캐시 적중률 등 API 비용 관련 메트릭을 추적해야 합니다.

기초 모델은 개방형 출력을 생성할 수 있기 때문에, 다양한 방식으로 문제가 발생할 수 있습니다. 메트릭 설계에는 분석적 사고, 통계 지식, 종종 창의성까지 요구됩니다. 어떤 메트릭을 추적해야 하는지는 애플리케이션마다 다릅니다.

이 책에서는 다양한 모델 품질 메트릭(4\~6장, 그리고 이 장의 뒷부분)과 이를 계산하는 다양한 방법(3장과 5장)을 다루었습니다. 여기서는 간략히 요약합니다.

추적하기 가장 쉬운 실패 유형은 포맷 오류입니다. 이런 오류는 쉽게 감지하고 검증할 수 있기 때문입니다. 예를 들어, JSON 출력을 기대할 경우, 모델이 잘못된 JSON을 얼마나 자주 출력하는지 추적하고, 그 중 몇 개가 쉽게 수정 가능한지를 기록합니다(예: 닫는 괄호 누락은 쉽게 수정 가능하지만, 예상 키가 누락된 경우는 어렵습니다).

개방형 생성에 대해서는 사실 일관성(factual consistency)과 관련 생성 품질 메트릭(간결성, 창의성, 긍정성 등)을 모니터링하는 것을 고려하세요. 이들 중 많은 메트릭은 AI 판별자(judge)를 사용하여 계산할 수 있습니다.

안전이 문제가 될 경우, 독성 관련 메트릭을 추적하고, 입력과 출력 모두에서 개인 및 민감 정보를 탐지하세요. 가드레일이 얼마나 자주 트리거되는지, 시스템이 얼마나 자주 응답을 거부하는지도 추적하세요. 비정상적인 쿼리도 탐지하세요. 이런 쿼리는 흥미로운 엣지 케이스나 프롬프트 공격을 드러낼 수 있습니다.

모델 품질은 사용자의 자연어 피드백이나 대화 신호를 통해서도 추론할 수 있습니다. 예를 들어, 추적 가능한 쉬운 메트릭에는 다음이 포함됩니다:

* 사용자가 생성을 중단하는 비율은 얼마나 되나요?
* 대화당 평균 턴 수는 얼마인가요?
* 입력당 평균 토큰 수는 얼마인가요? 사용자가 더 복잡한 작업에 애플리케이션을 사용하는 중인가요, 아니면 프롬프트를 더 간결하게 만드는 법을 배우는 중인가요?
* 출력당 평균 토큰 수는 얼마인가요? 어떤 모델이 더 장황한가요? 어떤 유형의 쿼리가 더 긴 답변을 유도하나요?
* 모델의 출력 토큰 분포는 어떤가요? 시간이 지남에 따라 어떻게 변했나요? 모델의 출력이 더 다양해졌나요, 덜 다양해졌나요?

길이 관련 메트릭은 지연 시간과 비용을 추적하는 데에도 중요합니다. 더 긴 컨텍스트와 응답은 일반적으로 지연 시간 증가 및 비용 상승을 유발합니다.

애플리케이션 파이프라인의 각 구성 요소는 자체 메트릭을 가질 수 있습니다. 예를 들어, RAG 애플리케이션에서는 검색 품질을 컨텍스트 관련성 및 컨텍스트 정밀도로 평가합니다. 벡터 데이터베이스는 데이터를 인덱싱하는 데 필요한 저장 공간과 쿼리 수행 시간으로 평가할 수 있습니다.

다수의 메트릭이 존재할 가능성이 높기 때문에, 이들 메트릭이 서로 어떻게 상관관계가 있는지를 측정하는 것이 유용합니다. 특히, DAU(일간 활성 사용자), 세션 시간(사용자가 애플리케이션에 능동적으로 참여한 시간), 구독자 수 등 비즈니스의 핵심 지표(north star)와의 상관관계를 분석해야 합니다. 핵심 지표와 강한 상관관계를 가진 메트릭은 핵심 지표 개선의 실마리를 줄 수 있으며, 전혀 상관이 없는 메트릭은 최적화 대상에서 제외할 수 있는 힌트를 줄 수 있습니다.

사용자 경험을 이해하기 위해 지연 시간을 추적하는 것은 필수입니다. 9장에서 다룬 일반적인 지연 시간 메트릭에는 다음이 포함됩니다:

* TTFT(Time to first token): 첫 번째 토큰이 생성되기까지 걸리는 시간
* TPOT(Time per output token): 각 출력 토큰 생성에 걸리는 시간
* 총 지연 시간: 응답 완료까지 걸리는 총 시간

이 모든 메트릭은 사용자별로 추적하여, 사용자 수 증가에 따른 시스템 확장성을 평가하세요.

비용도 추적해야 합니다. 비용 관련 메트릭에는 쿼리 수, 입력 및 출력 토큰의 양(예: 초당 토큰 수 TPS 등)이 포함됩니다. API 속도 제한이 있는 경우, 초당 요청 수도 추적하여 할당량 내에서 작동하는지 확인하고, 서비스 중단을 피해야 합니다.

메트릭을 계산할 때는 샘플링 방식(spot checks)과 전수 평가(exhaustive checks) 중 선택할 수 있습니다. 샘플링은 일부 데이터를 빠르게 분석하여 문제를 식별하고, 전수 평가는 모든 요청을 평가하여 전체 성능을 파악합니다. 시스템 요구사항과 사용 가능한 리소스에 따라 선택하며, 두 방식을 조합하여 균형 잡힌 모니터링 전략을 구축할 수 있습니다.

메트릭을 계산할 때, 사용자, 릴리스, 프롬프트/체인 버전, 프롬프트/체인 유형, 시간 등 관련 축에 따라 분해할 수 있어야 합니다. 이러한 세분화는 성능 차이를 이해하고 특정 문제를 식별하는 데 도움이 됩니다.
**Logs and traces**

메트릭은 일반적으로 집계된 형태입니다. 이는 시스템 내에서 시간 경과에 따라 발생하는 이벤트의 정보를 압축한 것으로, 시스템 상태를 한눈에 이해하는 데 도움이 됩니다. 그러나 메트릭만으로는 알 수 없는 질문들도 많습니다.

예를 들어, 특정 활동에 급증이 발생한 것을 본 후 "이런 일이 이전에도 있었나?"라는 의문이 생길 수 있습니다. 로그는 이러한 질문에 답하는 데 도움이 됩니다.

메트릭이 속성이나 이벤트를 수치적으로 측정한 것이라면, 로그는 이벤트에 대한 append-only(추가만 가능) 기록입니다. 프로덕션 환경에서 디버깅 과정은 다음과 같을 수 있습니다:

1. 메트릭은 5분 전에 무언가 잘못되었음을 알려주지만, 무슨 일이 발생했는지는 알려주지 않습니다.
2. 5분 전쯤 발생한 이벤트의 로그를 확인하여 무슨 일이 있었는지 파악합니다.
3. 로그의 오류와 메트릭을 연관 지어 문제를 올바르게 파악했는지 확인합니다.

빠른 탐지를 위해서는 메트릭이 신속히 계산되어야 하며, 빠른 대응을 위해서는 로그가 즉시 접근 가능해야 합니다. 로그가 15분 지연된다면, 5분 전에 발생한 문제를 추적하기 위해 로그가 도착할 때까지 기다려야 합니다.

앞으로 어떤 로그를 보게 될지 알 수 없기 때문에, 일반적인 로깅 규칙은 모든 것을 기록하는 것입니다. 모델 API 엔드포인트, 모델 이름, 샘플링 설정(temperature, top-p, top-k, 중단 조건 등), 프롬프트 템플릿 등 모든 설정을 로그에 기록하세요.

사용자 쿼리, 모델에 전달된 최종 프롬프트, 출력, 중간 출력도 기록하세요. 도구를 호출했는지 여부도 기록하세요. 도구의 출력도 기록하세요. 컴포넌트가 시작할 때, 끝날 때, 오류로 종료될 때 등을 기록하세요. 로그를 기록할 때는 이 로그가 시스템 내 어디에서 왔는지 알 수 있도록 태그와 ID를 반드시 포함하세요.

모든 것을 기록하면 로그의 양이 매우 빠르게 증가할 수 있습니다. 자동 로그 분석 및 로그 이상 탐지를 위한 많은 도구들이 AI를 활용합니다.

모든 로그를 수작업으로 처리하는 것은 불가능하지만, 프로덕션 데이터를 매일 수동으로 점검해보는 것은 사용자들이 애플리케이션을 어떻게 사용하는지를 파악하는 데 유용합니다. Shankar 외(2024)는 개발자들이 더 많은 데이터를 다루면서 좋은 출력과 나쁜 출력에 대한 인식이 변화하며, 이를 통해 프롬프트를 다시 작성해 좋은 응답의 가능성을 높이고 평가 파이프라인을 업데이트해 나쁜 응답을 포착하게 된다는 점을 발견했습니다.

로그가 서로 단절된 이벤트의 시퀀스라면, 트레이스(trace)는 연관된 이벤트를 연결하여 트랜잭션 또는 프로세스의 전체 타임라인을 재구성한 것입니다. 즉, 트레이스는 요청이 다양한 시스템 컴포넌트와 서비스를 거쳐 실행되는 경로를 상세히 기록한 것입니다. AI 애플리케이션에서는, 사용자가 쿼리를 보낸 시점부터 최종 응답을 받을 때까지의 전체 과정을 트레이싱할 수 있습니다. 여기에는 시스템이 수행한 작업, 검색된 문서, 모델에 전달된 최종 프롬프트 등이 포함됩니다. 각 단계에 걸린 시간과 측정 가능한 경우 비용도 표시되어야 합니다. 그림 10-11은 LangSmith에서 시각화한 요청의 트레이스를 보여줍니다.

이상적으로는, 각 쿼리가 시스템을 통과하며 어떻게 변형되는지 단계별로 추적할 수 있어야 합니다. 쿼리가 실패할 경우, 어느 단계에서 잘못되었는지를 정확히 파악할 수 있어야 합니다. 예를 들어, 잘못 처리되었는지, 검색된 컨텍스트가 관련이 없었는지, 모델이 잘못된 응답을 생성했는지 등을 알아야 합니다.

<img src="./images/fig_10_11.png" width=800>

Figure 10-11. LangSmith에서 시각화한 요청 트레이스.

**Drift detection**

시스템의 구성 요소가 많아질수록 변화할 수 있는 부분도 많아집니다. AI 애플리케이션에서는 다음과 같은 변화가 발생할 수 있습니다:

**System prompt changes**

애플리케이션의 시스템 프롬프트가 본인도 모르게 변경되는 경우는 많습니다. 시스템 프롬프트가 프롬프트 템플릿 위에 구성되어 있고, 해당 템플릿이 업데이트되었을 수 있습니다. 동료가 오타를 발견하고 수정했을 수도 있습니다. 간단한 로직만으로도 시스템 프롬프트가 변경되었는지를 감지할 수 있어야 합니다.

**User behavior changes**

시간이 지남에 따라 사용자는 기술에 적응하게 됩니다. 예를 들어, 사람들은 이미 더 나은 결과를 얻기 위해 Google 검색에서 쿼리를 구성하는 방법이나, 기사 순위를 올리는 방법을 터득했습니다. 자율주행차가 있는 지역에 사는 사람들은 자율주행차에게 길을 양보하게 만드는 법도 이미 익혔습니다(Liu 외, 2020). 당신의 사용자들도 애플리케이션에서 더 나은 결과를 얻기 위해 행동을 변화시킬 가능성이 있습니다. 예를 들어, 응답을 더 간결하게 만들기 위해 지침을 작성하는 법을 배울 수 있습니다. 이는 시간이 지남에 따라 응답 길이의 점진적인 감소로 이어질 수 있습니다. 메트릭만 보면 이러한 감소의 원인을 바로 알기 어렵기 때문에, 근본 원인을 이해하려면 별도의 조사 과정이 필요합니다.

**Underlying model changes**

> API를 통해 모델을 사용할 경우, API는 그대로지만 내부 모델이 업데이트될 수 있습니다. 4장에서 언급했듯이, 모델 제공자는 이러한 업데이트를 항상 공지하지 않으며, 모델 변경 사항을 감지하는 책임은 사용자에게 있습니다. 동일한 API의 서로 다른 버전은 성능에 큰 영향을 미칠 수 있습니다. 예를 들어, Chen 외(2023)는 GPT-4 및 GPT-3.5의 2023년 3월 버전과 6월 버전 간의 벤치마크 점수에 뚜렷한 차이가 있음을 관찰했습니다. 이와 유사하게, Voiceflow는 이전 GPT-3.5-turbo-0301에서 새로운 GPT-3.5-turbo-1106으로 변경하면서 성능이 10% 감소했다고 보고했습니다.

---


### AI Pipeline Orchestration

AI 애플리케이션은 여러 모델을 포함하고, 다양한 데이터베이스로부터 데이터를 검색하며, 폭넓은 도구에 접근하는 등 상당히 복잡해질 수 있습니다. 오케스트레이터는 이러한 다양한 구성 요소가 협력하여 종단 간 파이프라인을 구성할 수 있도록 명시하는 데 도움을 줍니다. 오케스트레이터는 구성 요소 간 데이터가 원활히 흐르도록 보장합니다. 높은 수준에서 오케스트레이터는 두 단계, 구성 요소 정의와 체이닝으로 작동합니다:

**Components definition**

오케스트레이터에게 시스템이 사용하는 구성 요소를 알려주어야 합니다. 여기에는 다양한 모델, 검색을 위한 외부 데이터 소스, 시스템이 사용할 수 있는 도구들이 포함됩니다. 모델 게이트웨이는 모델을 추가하는 과정을 더 쉽게 만들 수 있습니다. ${ }^{6}$ 평가 및 모니터링에 사용하는 도구가 있다면 그것들도 오케스트레이터에 알려줄 수 있습니다.

**Chaining**

체이닝은 기본적으로 함수 합성(function composition)입니다. 즉, 서로 다른 함수(구성 요소)를 결합하는 것입니다. 체이닝(또는 파이프라이닝)에서는 사용자 쿼리를 받은 이후 작업 완료까지 시스템이 거치는 단계를 오케스트레이터에게 지정합니다. 다음은 그 예시입니다:

1. 원시 쿼리를 처리합니다.
2. 처리된 쿼리를 기반으로 관련 데이터를 검색합니다.
3. 원래 쿼리와 검색된 데이터를 결합하여, 모델이 기대하는 형식의 프롬프트를 생성합니다.
4. 모델이 프롬프트를 기반으로 응답을 생성합니다.
5. 응답을 평가합니다.
6. 응답이 양호하다고 판단되면 사용자에게 반환합니다. 그렇지 않으면 쿼리를 인간 운영자에게 라우팅합니다.

오케스트레이터는 구성 요소 간 데이터를 전달하는 역할을 합니다. 현재 단계의 출력이 다음 단계가 기대하는 형식에 맞도록 도와주는 도구를 제공해야 합니다. 이상적으로는, 컴포넌트 실패나 데이터 형식 불일치와 같은 오류로 인해 데이터 흐름이 중단될 경우 이를 알려주어야 합니다.

**WARNING**

AI 파이프라인 오케스트레이터는 Airflow나 Metaflow와 같은 일반적인 워크플로 오케스트레이터와는 다릅니다.

지연 시간에 민감한 애플리케이션의 파이프라인을 설계할 때는 가능한 많은 작업을 병렬로 수행하는 것이 좋습니다. 예를 들어, 쿼리의 라우팅을 결정하는 컴포넌트와 PII 제거 컴포넌트가 있다면, 이 둘은 동시에 실행할 수 있습니다.

LangChain, LlamaIndex, Flowise, Langflow, Havstack 등 다양한 AI 오케스트레이션 도구들이 존재합니다. 검색 및 도구 사용은 흔한 애플리케이션 패턴이기 때문에, 많은 RAG 및 에이전트 프레임워크도 오케스트레이션 도구로 사용됩니다.

프로젝트를 시작할 때 바로 오케스트레이션 도구를 도입하고 싶은 유혹이 있을 수 있지만, 먼저 오케스트레이터 없이 애플리케이션을 구축해보는 것이 좋습니다. 외부 도구는 추가적인 복잡성을 수반하며, 오케스트레이터는 시스템이 작동하는 방식의 핵심적인 세부 사항을 추상화하여 시스템을 이해하거나 디버깅하기 어렵게 만들 수 있습니다.

애플리케이션 개발이 더 진행됨에 따라, 오케스트레이터가 작업을 더 쉽게 만들어줄 수 있다고 판단될 수 있습니다. 오케스트레이터를 평가할 때는 다음 세 가지 측면을 고려하세요:

**Integration and extensibility**

오케스트레이터가 이미 사용 중이거나 앞으로 도입할 수 있는 구성 요소를 지원하는지 평가하세요. 예를 들어, Llama 모델을 사용하고자 한다면, 해당 오케스트레이터가 이를 지원하는지 확인하세요. 수많은 모델, 데이터베이스, 프레임워크가 존재하기 때문에, 모든 것을 지원하는 오케스트레이터는 존재하지 않습니다. 따라서 확장성도 함께 고려해야 합니다. 특정 구성 요소를 지원하지 않을 경우, 그것을 추가하거나 바꾸는 것이 얼마나 어려운지 평가하세요.

**Support for complex pipelines**

애플리케이션이 복잡해짐에 따라, 여러 단계와 조건 분기를 포함한 복잡한 파이프라인을 관리해야 할 수도 있습니다. 분기(branching), 병렬 처리, 오류 처리와 같은 고급 기능을 지원하는 오케스트레이터는 이러한 복잡성을 효율적으로 관리하는 데 도움이 됩니다.

**Ease of use, performance, and scalability**

오케스트레이터의 사용 용이성을 고려하세요. 직관적인 API, 포괄적인 문서, 강력한 커뮤니티 지원이 있다면, 당신과 팀의 학습 곡선을 크게 줄일 수 있습니다. 숨겨진 API 호출을 유발하거나 애플리케이션에 지연을 추가하는 오케스트레이터는 피하세요. 또한, 애플리케이션 수, 개발자 수, 트래픽이 증가함에 따라 오케스트레이터가 효과적으로 확장 가능한지도 확인해야 합니다.

---

## **User Feedback**

사용자 피드백은 소프트웨어 애플리케이션에서 항상 두 가지 주요 방식으로 중요한 역할을 해왔습니다: 애플리케이션 성능 평가와 개발 방향 설정입니다. 그러나 AI 애플리케이션에서는 사용자 피드백이 훨씬 더 중요한 역할을 합니다. 사용자 피드백은 독점 데이터이며, 데이터는 경쟁 우위입니다. 8장에서 논의한 데이터 플라이휠을 만들기 위해서는 잘 설계된 사용자 피드백 시스템이 필요합니다. ${ }^{7}$

사용자 피드백은 개별 사용자를 위한 모델 맞춤화뿐만 아니라, 향후 모델 반복 학습에도 활용될 수 있습니다. 데이터가 점점 더 희소해짐에 따라, 독점 데이터의 가치는 그 어느 때보다도 커지고 있습니다. 제품을 빠르게 출시하고 초기에 사용자를 확보하면, 지속적으로 모델을 개선할 수 있는 데이터를 수집할 수 있으며, 이는 경쟁자가 따라잡기 어렵게 만듭니다.

사용자 피드백은 사용자 데이터라는 점을 잊지 말아야 합니다. 사용자 피드백을 활용하는 것은 다른 모든 데이터를 활용할 때와 동일한 주의가 필요합니다. 사용자 프라이버시는 반드시 존중되어야 하며, 사용자에게 그들의 데이터가 어떻게 사용되는지에 대한 알 권리가 보장되어야 합니다.

---

### Extracting Conversational Feedback

전통적으로 피드백은 명시적(explicit) 또는 암시적(implicit)일 수 있습니다. 명시적 피드백은 애플리케이션에서 피드백을 명시적으로 요청할 때 사용자가 제공하는 정보입니다. 예를 들어, 좋아요/싫어요, 업보트/다운보트, 별점 평가, 또는 "문제가 해결되었나요?"라는 질문에 대한 예/아니요 응답 등이 이에 해당합니다. 명시적 피드백은 애플리케이션 전반에서 상당히 표준화되어 있으며, 사람에게 어떤 것을 좋아하는지 묻는 방식은 한정되어 있기 때문에 명시적 피드백은 더 잘 이해됩니다.

암시적 피드백은 사용자 행동에서 유추한 정보입니다. 예를 들어, 사용자가 추천받은 상품을 구매했다면, 그것은 좋은 추천이었다는 것을 의미합니다. 무엇을 암시적 피드백으로 간주할지는 각 애플리케이션 내에서 사용자가 할 수 있는 행동에 따라 달라지며, 따라서 매우 애플리케이션 의존적입니다. 기본 모델은 새로운 종류의 애플리케이션을 가능하게 하며, 그에 따라 다양한 장르의 암시적 피드백도 생겨납니다.

많은 AI 애플리케이션이 사용하는 대화형 인터페이스는 사용자가 피드백을 더 쉽게 제공할 수 있게 합니다. 사용자는 일상 대화에서처럼 좋은 행동을 격려하고 오류를 수정하는 방식으로 피드백을 줄 수 있습니다. 사용자가 AI에게 지시를 내리는 방식 자체가 애플리케이션의 성능과 사용자 선호에 대한 피드백을 전달할 수 있습니다.

예를 들어, 당신이 AI 어시스턴트를 사용해 호주 여행을 계획하고 있다고 가정해 봅시다. 시드니에서 3박 동안 머물 호텔을 찾아달라고 요청했을 때, AI가 다음과 같은 세 가지 추천을 제공합니다:

> Here are three hotel suggestions in Sydney for a 3-night stay:

> 1. Historic Rocks Boutique Hotel (The Rocks)
>
> * Price: –\$400/night
> * Neighborhood: Charming streets and close to iconic sights.

> 2. Stylish Surry Hills House Hotel (Surry Hills)
>
> * Price: –\$200/night
> * Neighborhood: Trendy, with vibrant cafes and art galleries.

> 3. Chill Bondi Beachside Hotel (Bondi Beach)
>
> * Price: –\$300/night
> * Neighborhood: Beachside, ideal for relaxation and city exploration.

이 세 가지 추천에 대한 사용자의 응답은 그 사람의 선호도를 보여줍니다. 예를 들어, “갤러리 근처에 있는 곳으로 예약해줘”라고 응답한다면 예술에 대한 관심을 나타냅니다. 반면 “\$200 이하인 곳은 없어?”라는 응답은 가격에 민감한 성향을 드러내며, 어시스턴트가 아직 사용자를 잘 이해하지 못했음을 암시합니다.

대화에서 추출된 사용자 피드백은 다음과 같은 목적으로 활용될 수 있습니다:

* 평가: 애플리케이션을 모니터링하기 위한 메트릭 도출
* 개발: 미래 모델을 학습시키거나 개발 방향을 안내
* 개인화: 각 사용자에 맞춰 애플리케이션을 개인화

암시적 대화 피드백은 사용자 메시지의 내용과 의사소통 방식 모두에서 유추할 수 있습니다. 피드백이 일상 대화에 녹아 있기 때문에, 이를 추출하는 것은 도전적인 일입니다. 대화 단서에 대한 직관이 초기 신호 세트를 고안하는 데 도움이 될 수 있지만, 엄밀한 데이터 분석과 사용자 연구가 필요합니다.

대화형 피드백은 챗봇의 인기에 힘입어 더 많은 주목을 받고 있지만, ChatGPT가 등장하기 이전에도 수년간 활발히 연구되던 주제였습니다. 강화 학습 커뮤니티에서는 2010년대 후반부터 자연어 피드백을 통해 RL 알고리즘이 학습할 수 있도록 노력해왔으며, 많은 연구들이 유망한 결과를 보였습니다(Fu et al., 2019; Goyal et al., 2019; Zhou and Small, 2020; Sumers et al., 2020). 자연어 피드백은 Amazon Alexa(Ponnusamy et al., 2019; Park et al., 2020), Spotify 음성 제어(Xiao et al., 2021), Yahoo! Voice(Hashimoto and Sassano, 2018)와 같은 초기 대화형 AI 애플리케이션에서도 큰 관심을 받아왔습니다.

**Natural language feedback**

메시지의 내용에서 추출된 피드백을 자연어 피드백이라고 합니다. 다음은 대화 진행 상황을 알려주는 자연어 피드백 신호 몇 가지입니다. 프로덕션 환경에서는 애플리케이션 성능을 모니터링하기 위해 이러한 신호들을 추적하는 것이 유용합니다.

* **조기 종료**

  * 사용자가 응답 생성을 중단하거나(예: 생성 중간에 멈춤), 앱에서 나가거나(웹/모바일 앱의 경우), 모델에게 그만하라고 지시하거나(음성 어시스턴트의 경우), 또는 응답 옵션을 선택하지 않고 떠나는 경우 등은 대화가 잘 진행되지 않았음을 나타냅니다.
* **오류 수정**

  * 사용자가 후속 발화를 “아니, …” 또는 “내 말은, …”으로 시작하면, 모델의 응답이 빗나갔을 가능성이 높습니다.
  * 오류를 수정하기 위해, 사용자는 요청을 다시 표현하려 할 수 있습니다. 그림 10-12는 사용자가 모델의 오해를 바로잡기 위해 질문을 다시 표현하는 시도의 예시를 보여줍니다. 재표현 시도는 휴리스틱이나 머신러닝 모델을 사용해 감지할 수 있습니다.

<img src="./images/fig_10_12.png" width=800>

Figure 10-12. 사용자가 생성 중단과 질문 재표현을 모두 수행했기 때문에, 모델이 원래 요청의 의도를 오해한 것으로 추론할 수 있습니다.

사용자는 또한 모델이 달리 했어야 했던 특정 사항을 지적할 수도 있습니다. 예를 들어, 사용자가 모델에게 이야기를 요약해달라고 요청했고 모델이 인물을 혼동한 경우, 사용자는 “빌은 용의자지, 피해자가 아니야”라고 피드백을 줄 수 있습니다. 모델은 이 피드백을 받아 요약을 수정할 수 있어야 합니다.

이런 종류의 행동 수정 피드백은 에이전트 기반 사용 사례에서 특히 흔하며, 사용자가 에이전트를 보다 바람직한 방향으로 유도할 수 있도록 합니다. 예를 들어, 사용자가 회사 XYZ에 대한 시장 분석 작업을 에이전트에게 맡기면, “XYZ의 GitHub 페이지도 확인해봐” 또는 “CEO의 X 프로필도 확인해봐”와 같은 피드백을 줄 수 있습니다.

때때로 사용자는 “확실해?”, “다시 확인해봐”, “출처를 보여줘”와 같은 명시적 확인을 요청함으로써 모델이 스스로를 수정하길 원할 수도 있습니다. 이는 반드시 모델이 잘못된 답변을 했다는 의미는 아니지만, 모델의 응답이 사용자가 원하는 세부 정보를 결여했거나 모델에 대한 일반적인 불신을 나타낼 수 있습니다.

일부 애플리케이션은 사용자가 모델의 응답을 직접 수정할 수 있도록 허용합니다. 예를 들어, 사용자가 모델에게 코드를 생성하도록 요청했고, 그 코드를 수정했다면, 이는 생성된 코드가 완전히 맞지 않았다는 강력한 신호입니다.

사용자 수정은 또한 선호도(preference) 데이터의 귀중한 원천입니다. 선호도 데이터는 일반적으로 (쿼리, 우세 응답, 열세 응답)의 형식을 가지며, 모델을 인간의 선호에 맞춰 정렬하는 데 사용됩니다. 각 사용자 수정은 하나의 선호도 예시를 구성하며, 원래 생성된 응답은 열세 응답이 되고, 수정된 응답은 우세 응답이 됩니다.
**Complaints**

종종 사용자는 애플리케이션의 출력에 대해 수정하려 하기보다는 단순히 불만을 표현합니다. 예를 들어, 답변이 틀렸거나, 관련성이 없거나, 독성이 있거나, 너무 길거나, 세부 정보가 부족하거나, 그냥 형편없다고 불평할 수 있습니다. 표 10-1은 FITS(Feedback for Interactive Talk & Search) 데이터셋(Xu et al., 2022)을 자동 클러스터링하여 도출한 여덟 가지 자연어 피드백 유형을 보여줍니다.

Table 10-1. FITS 데이터셋(Xu et al. 및 Yuan et al. (2023))을 자동 클러스터링하여 도출한 피드백 유형.

| Group |                                 Feedback type                                 | Num. |
| :---: | :---------------------------------------------------------------------------: | :--: |
|   1   |                               다시 요구사항을 명확히 설명함.                               | 3702 |
|   2   | 챗봇이 (1) 질문에 답하지 않거나, (2) 관련 없는 정보를 제공하거나, (3) 사용자에게 스스로 답을 찾으라고 하는 것에 대해 불평함. | 2260 |
|   3   |                           질문에 답할 수 있는 특정 검색 결과를 지적함.                          | 2255 |
|   4   |                            챗봇이 검색 결과를 사용해야 한다고 제안함.                           | 2130 |
|   5   |                  응답이 (1) 사실과 다르거나, (2) 검색 결과에 기반하지 않는다고 주장함.                  | 1572 |
|   6   |                      챗봇의 답변이 구체성/정확성/완전성/세부성이 부족하다고 지적함.                      | 1309 |
|   7   |             챗봇이 자신감이 없으며 항상 "확실하지 않아요" 또는 "모르겠어요"로 응답을 시작한다고 지적함.             |  582 |
|   8   |                           챗봇 응답에서 반복 또는 무례함에 대해 불평함.                          |  137 |

챗봇이 사용자에게 어떻게 실패하는지를 이해하는 것은 개선을 위한 핵심입니다. 예를 들어, 사용자가 장황한 응답을 싫어한다는 사실을 알게 되면, 챗봇의 프롬프트를 더 간결하게 조정할 수 있습니다. 사용자가 응답이 세부 정보가 부족하다고 불만을 표현했다면, 챗봇이 더 구체적인 답변을 하도록 유도할 수 있습니다.

**Sentiment**

불만은 이유를 설명하지 않고 단지 부정적인 감정(좌절, 실망, 비꼼 등)을 표현하는 것일 수도 있습니다. 예: "으악(Uggh)". 이는 다소 암울하게 들릴 수 있지만, 사용자가 챗봇과의 대화에서 보이는 감정 분석은 챗봇의 성능에 대한 통찰을 줄 수 있습니다. 일부 콜센터는 통화 중 사용자 음성의 감정을 추적합니다. 사용자가 점점 더 목소리를 높인다면, 무언가 잘못되고 있는 것입니다. 반대로, 화난 상태로 대화를 시작했지만 마지막에 행복해졌다면, 문제가 해결되었을 가능성이 있습니다.

자연어 피드백은 모델의 응답에서 유추할 수도 있습니다. 중요한 신호 중 하나는 모델의 거절률(refusal rate)입니다. 모델이 "죄송하지만, 그건 잘 모르겠어요" 또는 "저는 언어 모델이라 ... 할 수 없습니다"와 같은 표현을 자주 사용한다면, 사용자가 만족하지 못했을 가능성이 높습니다.

**Other conversational feedback**

다른 유형의 대화형 피드백은 메시지 대신 사용자 행동에서 파생될 수 있습니다.

**Regeneration**

많은 애플리케이션은 사용자가 다른 응답을 생성할 수 있도록 허용하며, 때로는 다른 모델을 사용하기도 합니다. 사용자가 재생성을 선택하는 이유는 첫 번째 응답에 만족하지 않아서일 수도 있고, 첫 번째 응답이 적절하더라도 비교할 수 있는 옵션을 원해서일 수도 있습니다. 이는 이미지 생성이나 이야기 생성과 같은 창의적인 요청에서 특히 흔합니다.

재생성 신호는 종량제 요금제를 사용하는 애플리케이션에서 구독 기반 애플리케이션보다 더 강력할 수 있습니다. 종량제에서는 사용자가 단순한 호기심 때문에 추가 비용을 감수하고 재생성을 시도할 가능성이 낮습니다.

개인적으로, 복잡한 요청에 대해서는 모델 응답의 일관성을 확인하기 위해 재생성을 자주 선택합니다. 두 응답이 서로 모순된 답변을 제공한다면, 둘 다 신뢰할 수 없습니다.

재생성 후, 일부 애플리케이션은 그림 10-13에서와 같이 새로운 응답을 이전 응답과 비교하도록 명시적으로 요청하기도 합니다. 이러한 "더 나음/덜 나음" 데이터는 선호도 파인튜닝에 다시 활용될 수 있습니다.

<img src="./images/fig_10_13.png" width=800>

Figure 10-13. 사용자가 다른 응답을 생성한 후, ChatGPT는 비교 피드백을 요청함.

**Conversation organization**

사용자가 대화를 정리하기 위해 수행하는 행동—예: 삭제, 이름 변경, 공유, 북마크—도 신호가 될 수 있습니다. 대화를 삭제하는 것은 대화가 형편없었다는 강력한 신호일 수 있지만, 부끄러운 대화를 없애기 위한 것일 수도 있습니다. 대화의 이름을 바꾸는 것은 대화가 좋았지만, 자동 생성된 제목이 적절하지 않았다는 것을 나타냅니다.

**Conversation length**

또 다른 일반적인 추적 신호는 대화당 턴 수입니다. 이 신호가 긍정적인지 부정적인지는 애플리케이션에 따라 달라집니다. AI 동반자 앱에서는 긴 대화가 사용자가 대화를 즐기고 있다는 신호일 수 있습니다. 반면, 고객 지원과 같은 생산성 중심 챗봇에서는 긴 대화가 봇이 사용자의 문제를 해결하는 데 비효율적이라는 신호일 수 있습니다.

**Dialogue diversity**

대화 길이는 대화 다양성(dialogue diversity)과 함께 해석될 수도 있습니다. 대화 다양성은 고유 토큰 수나 주제 수로 측정할 수 있습니다. 예를 들어, 대화가 길지만 챗봇이 동일한 문장을 반복한다면, 사용자가 루프에 갇힌 상태일 수 있습니다.

명시적 피드백은 해석이 쉽지만, 사용자에게 추가적인 노력을 요구합니다. 많은 사용자가 이 추가 작업을 기꺼이 하지 않기 때문에, 명시적 피드백은 특히 사용자 기반이 작은 애플리케이션에서 부족할 수 있습니다. 또한 명시적 피드백은 응답 편향(response bias)의 영향을 받을 수 있습니다. 예를 들어, 불만이 있는 사용자가 더 적극적으로 피드백을 제공하는 경향이 있어 실제보다 부정적으로 보일 수 있습니다.

암시적 피드백은 훨씬 풍부하며, 어떤 것이 암시적 피드백이 될 수 있는지는 상상력에 달려 있습니다. 그러나 더 많은 잡음을 동반합니다. 암시적 신호를 해석하는 것은 어려울 수 있습니다. 예를 들어, 대화를 공유하는 것이 부정적인 신호일 수도 있고 긍정적인 신호일 수도 있습니다. 예를 들어, 어떤 친구는 모델이 심각한 실수를 했을 때 대화를 공유하고, 다른 친구는 유용한 대화를 동료들과 공유합니다. 사용자가 왜 특정 행동을 했는지를 이해하기 위해 사용자를 연구하는 것이 중요합니다.

더 많은 신호를 추가하면 의도를 더 명확히 이해하는 데 도움이 될 수 있습니다. 예를 들어, 사용자가 링크를 공유한 뒤 질문을 재표현한다면, 이는 해당 대화가 기대에 미치지 못했다는 신호일 수 있습니다. 대화에서 암시적 응답을 추출하고, 해석하고, 활용하는 것은 아직 작지만 성장 중인 연구 분야입니다.

---

### Feedback Design

어떤 피드백을 수집해야 할지 확신이 없었다면, 앞선 섹션이 아이디어를 제공했기를 바랍니다.

이번 섹션에서는 이 귀중한 피드백을 언제, 어떻게 수집할지를 다룹니다.

**When to collect feedback**

피드백은 사용자 여정 전반에서 수집할 수 있으며 그래야 합니다. 특히 오류를 보고해야 할 필요가 생겼을 때, 사용자는 피드백을 제공할 수 있어야 합니다. 그러나 피드백 수집 옵션은 방해가 되지 않아야 합니다. 사용자 워크플로우를 방해해서는 안 됩니다. 다음은 사용자 피드백이 특히 가치 있을 수 있는 몇 가지 지점입니다.

**초기 단계에서**

사용자가 가입한 직후, 사용자 피드백은 애플리케이션을 사용자에게 맞춰 조정하는 데 도움이 될 수 있습니다. 예를 들어, 얼굴 인식 앱은 먼저 얼굴을 스캔해야 작동합니다. 음성 어시스턴트는 웨이크 워드(예: "Hey Google") 인식을 위해 사용자가 문장을 읽도록 요청할 수 있습니다. 언어 학습 앱은 사용자의 실력을 파악하기 위해 몇 가지 질문을 할 수 있습니다. 얼굴 인식처럼 보정이 필수인 애플리케이션도 있고, 그렇지 않은 경우 초기 피드백은 선택적으로 제공되어야 합니다. 이는 사용자가 제품을 체험하는 데 마찰을 줄이기 위함입니다. 사용자가 선호도를 명시하지 않으면, 중립 옵션으로 시작하고 시간이 지남에 따라 조정할 수 있습니다.

**문제가 발생했을 때**

모델이 환각된 응답을 생성하거나, 정상적인 요청을 차단하거나, 부적절한 이미지를 생성하거나, 응답이 너무 느릴 경우, 사용자는 이러한 실패를 보고할 수 있어야 합니다. 사용자는 응답에 대해 다운보트를 누르거나, 동일한 모델로 다시 생성하거나, 다른 모델로 전환하는 등의 옵션을 가질 수 있습니다. 또는 "틀렸어요", "너무 뻔해요", "좀 더 짧게 해줘요" 같은 대화 피드백을 제공할 수도 있습니다.

이상적으로는 제품이 실수를 하더라도 사용자가 여전히 작업을 완료할 수 있어야 합니다. 예를 들어, 모델이 제품을 잘못 분류했을 경우, 사용자가 직접 카테고리를 수정할 수 있어야 합니다. 사용자에게 AI와 협업할 수 있는 수단을 제공하세요. 그래도 안 되면, 인간과 협업할 수 있게 해야 합니다. 많은 고객 지원 봇은 대화가 길어지거나 사용자가 좌절하는 듯 보이면 인간 상담원에게 전환하는 옵션을 제공합니다.

이미지 생성에서의 예시는 인페인팅(inpainting) 기능입니다. \${ }^{9}\$ 생성된 이미지가 사용자의 요구에 정확히 맞지 않을 경우, 사용자는 이미지의 특정 영역을 선택하고, 해당 영역을 어떻게 개선할지 프롬프트로 설명할 수 있습니다. 그림 10-14는 OpenAI의 DALL-E에서 인페인팅이 작동하는 예시를 보여줍니다. 이 기능은 사용자가 더 나은 결과를 얻을 수 있도록 하며, 개발자에게는 고품질 피드백을 제공합니다.

<img src="./images/fig_10_14.png" width=800>

Figure 10-14. DALL-E에서 인페인팅이 작동하는 예시. 이미지 출처: OpenAI.

**모델이 확신이 없을 때**

모델이 특정 작업에 대해 확신이 없을 경우, 사용자에게 피드백을 요청하여 확신을 높일 수 있습니다. 예를 들어, 논문 요약 요청을 받았지만 사용자가 간단한 개요를 원하는지, 섹션별 세부 요약을 원하는지 확실하지 않을 경우, 모델은 두 가지 요약을 나란히 출력할 수 있습니다. 단, 두 개의 요약을 출력해도 사용자에게 지연을 유발하지 않는다면 말이죠. 사용자는 선호하는 쪽을 선택할 수 있으며, 이와 같은 비교 피드백은 선호도 파인튜닝에 활용할 수 있습니다. 그림 10-15는 프로덕션 환경에서의 비교 평가 예시입니다.

<img src="./images/fig_10_15.png" width=800>

Figure 10-15. ChatGPT 응답 두 개를 나란히 비교하여 피드백을 수집하는 예시.

사용자에게 두 개의 전체 응답을 읽고 선택하게 한다는 것은 명시적 피드백을 요청하는 것입니다. 그러나 사용자는 두 응답을 읽을 시간이나 그럴 의지가 없을 수도 있으며, 이로 인해 피드백은 잡음이 많을 수 있습니다. Google Gemini와 같은 일부 애플리케이션은 각 응답의 시작 부분만 보여줍니다(그림 10-16 참조). 사용자는 더 읽고 싶은 응답을 클릭해야 하며, 이는 어떤 응답이 더 유망하게 보이는지에 대한 피드백을 제공합니다. 단, 전체 응답을 보여주는 것이 더 신뢰할 수 있는 피드백을 유도하는지, 일부만 보여주는 것이 더 나은지는 아직 명확하지 않습니다. \${ }^{10}\$

<img src="./images/fig_10_16.png" width=800>

Figure 10-16. Google Gemini는 비교 피드백을 위해 응답의 일부만 나란히 보여주며, 사용자는 더 보고 싶은 응답을 클릭해야 함.

또 다른 예시는 사진을 자동으로 태그해주는 사진 정리 애플리케이션입니다. "X가 나온 모든 사진 보여줘"와 같은 쿼리에 응답하기 위해, 두 인물이 같은 사람인지 확실하지 않을 경우, Google Photos처럼 사용자에게 피드백을 요청할 수 있습니다(그림 10-17).

<img src="./images/fig_10_17.png" width=800>

Figure 10-17. Google Photos는 확신이 없을 때 사용자 피드백을 요청함. 두 고양이 이미지는 ChatGPT가 생성함.

"그렇다면 좋은 결과가 나왔을 때 피드백은 어떻게 해야 할까?"라는 의문이 생길 수 있습니다. 사용자가 만족을 표현할 수 있는 행동에는 좋아요, 즐겨찾기, 공유 등이 포함됩니다. 그러나 Apple의 휴먼 인터페이스 가이드라인은 긍정적 피드백과 부정적 피드백을 모두 요청하는 것을 경고합니다. 애플리케이션은 기본적으로 좋은 결과를 제공해야 하며, 좋은 결과에 대해 피드백을 요청하면 사용자가 좋은 결과를 예외로 인식할 수 있습니다.

하지만 내가 이야기한 많은 사람들은, 놀라운 경험을 했을 때 사용자에게 피드백 기회를 제공해야 한다고 믿습니다. 인기 있는 AI 기반 제품의 한 제품 매니저는, 그들의 팀이 긍정적인 피드백을 필요로 한다고 말했습니다. 왜냐하면 그것이 사용자들이 열정적으로 피드백을 줄 만큼 좋아하는 기능을 드러내기 때문입니다. 이는 팀이 최소한의 가치를 더하는 여러 기능에 자원을 분산시키는 대신, 높은 임팩트를 주는 기능 몇 개에 집중하도록 도와줍니다.

일부는 긍정적 피드백 요청이 인터페이스를 복잡하게 하거나 사용자에게 짜증을 유발할까 봐 이를 피합니다. 그러나 이러한 위험은 피드백 요청 빈도를 제한함으로써 관리할 수 있습니다. 예를 들어, 사용자 기반이 큰 경우, 사용자 중 단 1%에게만 피드백 요청을 보여주는 것으로도 충분한 피드백을 수집하면서 대부분의 사용자 경험을 방해하지 않을 수 있습니다. 물론 요청 대상 비율이 작을수록 피드백 편향의 위험은 커지지만, 사용자 풀이 충분히 크다면 의미 있는 제품 인사이트를 제공할 수 있습니다.
**피드백을 수집하는 방법**

피드백은 사용자의 워크플로에 자연스럽게 통합되어야 합니다. 사용자가 추가적인 작업 없이 쉽게 피드백을 제공할 수 있어야 합니다. 피드백 수집은 사용자 경험을 방해해서는 안 되며, 무시하기 쉬워야 합니다. 사용자에게 좋은 피드백을 제공할 인센티브도 있어야 합니다.

종종 좋은 피드백 설계의 예로 언급되는 것은 이미지 생성 앱 Midjourney입니다. 각 프롬프트에 대해 Midjourney는 네 개의 이미지를 생성하고, 사용자에게 그림 10-18에 표시된 다음과 같은 옵션을 제공합니다:

1. 이들 이미지 중 하나의 비축소 버전을 생성합니다.
2. 이들 이미지 중 하나에 대해 변형을 생성합니다.
3. 다시 생성합니다.

이러한 모든 옵션은 Midjourney에 서로 다른 신호를 제공합니다. 옵션 1과 2는 사용자가 네 개의 사진 중 어떤 것이 가장 유망하다고 생각하는지를 Midjourney에 알려줍니다. 옵션 1은 선택된 사진에 대해 가장 강한 긍정적인 신호를 줍니다. 옵션 2는 더 약한 긍정적인 신호를 줍니다. 옵션 3은 네 장의 사진 모두 충분히 좋지 않다는 신호를 줍니다. 그러나 사용자는 기존의 사진이 좋아도 다른 것이 가능한지 보기 위해 다시 생성을 선택할 수도 있습니다.

Zootopia 스타일의 경찰 제복을 입은, 모험을 함께 떠나고 싶어지는 웃으며 윙크하는 라마의 클로즈업 사진 - @chiphuyen (fast)

<img src="./images/fig_10_18.png" width=800>

그림 10-18. Midjourney의 워크플로는 앱이 암묵적 피드백을 수집할 수 있도록 해줍니다.

GitHub Copilot과 같은 코드 어시스턴트는 그림 10-19에 표시된 것처럼 제안된 초안을 최종 텍스트보다 더 흐린 색상으로 표시할 수 있습니다. 사용자는 Tab 키를 눌러 제안을 수락하거나, 계속 타이핑하여 제안을 무시할 수 있으며, 이 둘 모두 피드백을 제공합니다.

<img src="./images/fig_10_19.png" width=800>

그림 10-19. GitHub Copilot은 제안을 수락하거나 거절하기 쉽게 만들어 줍니다.

ChatGPT나 Claude 같은 독립 실행형 AI 애플리케이션의 가장 큰 과제 중 하나는 사용자의 일상적인 워크플로에 통합되어 있지 않다는 점입니다. 이로 인해 GitHub Copilot과 같은 통합 제품에서처럼 고품질 피드백을 수집하기 어렵습니다. 예를 들어, Gmail이 이메일 초안을 제안하는 경우, Gmail은 이 초안이 어떻게 사용되었는지 또는 수정되었는지를 추적할 수 있습니다. 그러나 ChatGPT를 사용해 이메일을 작성하면, 이 이메일이 실제로 전송되었는지 여부를 ChatGPT는 알 수 없습니다.

피드백 자체만으로도 제품 분석에 도움이 될 수 있습니다. 예를 들어, 단순히 좋아요/싫어요 정보만 보더라도, 사람들이 얼마나 자주 제품에 만족하거나 불만족하는지를 계산하는 데 유용합니다. 그러나 더 깊은 분석을 위해서는 피드백에 대한 맥락이 필요합니다. 예를 들어, 피드백이 수집되기 전의 5\~10개의 대화 턴이 필요할 수 있습니다. 이 맥락은 무엇이 잘못되었는지를 파악하는 데 도움이 됩니다. 그러나 이 맥락에는 개인 식별 정보가 포함될 수 있기 때문에, 명시적인 사용자 동의 없이 이를 확보하는 것은 불가능할 수 있습니다.

이러한 이유로, 일부 제품은 서비스 약관에 제품 개선 및 분석을 위한 사용자 데이터 접근을 허용하는 조항을 포함시킵니다. 이러한 조항이 없는 애플리케이션의 경우, 사용자 피드백은 피드백과 함께 최근 상호작용 데이터를 공유하도록 요청하는 사용자 데이터 기부 흐름에 연결될 수 있습니다. 예를 들어, 피드백을 제출할 때 최근 데이터도 함께 공유할지를 묻는 체크박스를 표시할 수 있습니다.

사용자에게 피드백이 어떻게 사용되는지를 설명하면, 사용자들이 더 많은, 더 나은 피드백을 제공하도록 유도할 수 있습니다. 사용자의 피드백이 제품을 개인화하는 데 사용되는가, 일반 사용 통계를 수집하는 데 사용되는가, 아니면 새로운 모델을 학습하는 데 사용되는가? 사용자가 개인정보 보호에 민감한 경우, 데이터가 모델 학습에 사용되지 않거나 장치를 벗어나지 않는다는 점을 (해당 사항이 사실일 때만) 안심시켜야 합니다.

사용자에게 불가능한 일을 요구하지 마세요. 예를 들어, 사용자로부터 비교 신호를 수집하는 경우, 사용자가 이해하지 못하는 두 가지 옵션 중 하나를 선택하도록 요구하지 마세요. 예를 들어, 나는 ChatGPT가 그림 10-20에서 통계 질문에 대한 두 개의 가능한 답변 중 하나를 선택하라고 했을 때 당황한 적이 있습니다. "모르겠습니다"라는 옵션이 있었으면 좋았을 것입니다.

<img src="./images/fig_10_20.png" width=800>

그림 10-20. ChatGPT가 사용자에게 선호하는 응답을 선택하라고 요청한 예시. 그러나 이런 수학 문제는 선호의 문제가 되어서는 안 됩니다.

아이콘과 툴팁이 옵션을 이해하는 데 도움이 된다면, 이를 추가하세요. 사용자에게 혼동을 줄 수 있는 디자인은 피하세요. 모호한 지침은 잡음이 많은 피드백으로 이어질 수 있습니다. 나는 한 번 Luma를 사용해 GPU 최적화 워크숍을 진행하며 피드백을 수집했는데, 부정적인 피드백을 읽고 혼란스러웠습니다. 응답 내용은 긍정적이었지만, 별점은 \$1 / 5\$였습니다. 자세히 조사해보니, Luma는 피드백 수집 폼에서 숫자를 이모지로 표현했는데, 별 다섯 개짜리 평가가 되어야 할 자리에 분노 이모지(별 하나짜리 평가에 해당됨)를 배치한 것이었습니다. 그림 10-21을 참조하세요.

피드백을 공개할지 비공개할지에 대해 신중하게 고려하세요. 예를 들어, 사용자가 어떤 것을 좋아한다고 표시할 경우, 이 정보를 다른 사용자에게도 보여줄 것인지 생각해 보세요. Midjourney의 초기에는, 이미지를 확대하거나 변형을 생성하거나 새로운 배치를 재생성하는 것과 같은 피드백이 공개되었습니다.

<img src="./images/fig_10_21.png" width=800>

그림 10-21. Luma가 별 다섯 개짜리 평가가 되어야 할 자리에 별 하나짜리에 해당하는 분노 이모지를 배치했기 때문에, 일부 사용자는 실수로 이를 긍정적인 평가로 선택했습니다.

신호의 가시성은 사용자 행동, 사용자 경험, 피드백 품질에 깊은 영향을 줄 수 있습니다. 사용자는 비공개 상태에서 더 솔직해지는 경향이 있으며, 이는 더 높은 품질의 신호로 이어질 수 있습니다. 2024년, X(구 Twitter)는 "좋아요"를 비공개로 전환했습니다. X의 소유주 Elon Musk는 이 변경 이후 "좋아요" 수가 크게 증가했다고 주장했습니다.

그러나 비공개 신호는 콘텐츠의 발견 가능성과 설명 가능성을 줄일 수 있습니다. 예를 들어, "좋아요"를 숨기면 사용자가 친구들이 좋아한 트윗을 찾을 수 없게 됩니다. 만약 X가 사용자가 팔로우하는 사람들이 누른 좋아요를 기반으로 트윗을 추천한다면, "좋아요"가 숨겨졌을 경우 사용자는 왜 특정 트윗이 자신의 피드에 나타나는지 혼란스러울 수 있습니다.

---


### Feedback Limitation

애플리케이션 개발자에게 사용자 피드백의 가치에는 의심의 여지가 없습니다. 하지만 피드백은 공짜 점심이 아닙니다. 그것은 자체적인 한계를 가지고 있습니다.

**편향(Biases)**

다른 데이터와 마찬가지로, 사용자 피드백에도 편향이 있습니다. 이러한 편향을 이해하고 피드백 시스템을 이에 맞게 설계하는 것이 중요합니다. 각 애플리케이션은 고유의 편향을 가집니다. 다음은 주의해야 할 몇 가지 피드백 편향의 예입니다:

**관대성 편향(Leniency bias)**

관대성 편향은 사람들이 너무 긍정적으로 평가하려는 경향으로, 이는 종종 갈등을 피하려 하거나 친절하게 행동해야 한다는 느낌, 혹은 그것이 가장 쉬운 선택이기 때문입니다. 예를 들어, 당신이 급한 상황에서 앱이 거래를 평가하라고 요구할 때, 거래에 만족하지 않더라도 부정적인 평가를 하면 이유를 설명해야 한다는 걸 알고 있어서 그냥 긍정적으로 선택하고 끝내는 경우입니다. 이것이 피드백을 위해 사용자에게 추가적인 일을 시켜서는 안 되는 이유이기도 합니다.

별 다섯 개 평가 척도에서, 네 개와 다섯 개의 별은 일반적으로 좋은 경험을 의미합니다. 그러나 많은 경우, 사용자는 다섯 개의 별을 주도록 압박을 느끼며, 문제가 생겼을 때만 네 개를 선택합니다. Uber에 따르면, 2015년에 평균 운전기사 평점은 4.8이었으며, 4.6 이하의 점수는 비활성화 위험을 의미했습니다.

이러한 편향이 반드시 치명적인 문제는 아닙니다. Uber의 목표는 좋은 운전자와 나쁜 운전자를 구분하는 것입니다. 이 편향이 있음에도 불구하고, Uber의 평점 시스템은 이 목표를 달성하는 데 도움이 되는 것으로 보입니다. 사용자 평점의 분포를 살펴보면 이러한 편향을 감지할 수 있습니다.

더 세분화된 피드백을 원한다면, 낮은 평점에 대한 강한 부정적인 의미를 제거하면 사람들이 이 편향에서 벗어날 수 있도록 도울 수 있습니다. 예를 들어, 사용자에게 숫자 1\~5를 보여주는 대신 다음과 같은 옵션을 보여주세요:

> * “좋은 탑승이었어요. 훌륭한 기사님입니다.”
>
> * “꽤 괜찮았어요.”
>
> * “불만은 없지만 특별히 좋지도 않았어요.”
>
> * “좀 더 나았으면 좋았겠어요.”
>
> * “이 기사님과는 다시 매칭되지 않기를 바랍니다.”^{12}

**무작위성(Randomness)**

사용자들은 종종 악의 없이 무작위로 피드백을 제공합니다. 그 이유는 더 사려 깊은 입력을 제공할 동기가 없기 때문입니다. 예를 들어, 비교 평가를 위해 두 개의 긴 응답이 나란히 표시되었을 때, 사용자는 둘 다 읽기 싫어 그냥 아무거나 클릭할 수 있습니다. Midjourney의 경우, 사용자들이 단순히 무작위로 이미지를 선택해 변형을 생성할 수도 있습니다.

**위치 편향(Position bias)**

사용자에게 옵션이 제시되는 위치는 그 옵션이 어떻게 인식되는지에 영향을 줍니다. 일반적으로 사용자는 두 번째 옵션보다 첫 번째 제안을 클릭할 가능성이 높습니다. 사용자가 첫 번째 제안을 클릭했다고 해서 그것이 좋은 제안이라는 의미는 아닙니다.

피드백 시스템을 설계할 때 이 편향은 제안의 위치를 무작위로 바꾸거나, 위치에 따른 제안의 실제 성공률을 계산하는 모델을 구축함으로써 완화할 수 있습니다.

**선호 편향(Preference bias)**

사람의 피드백에 영향을 미치는 다른 많은 편향이 존재합니다. 이 책에서 다룬 것들 중 일부입니다. 예를 들어, 사람들은 두 응답을 비교할 때 길이가 더 긴 응답을 더 선호할 수 있습니다. 이는 길이가 정확성보다 인지하기 쉬운 특징이기 때문입니다. 또 다른 예는 최신성 편향(recency bias)으로, 사람들이 두 개의 응답 중 나중에 본 응답을 더 선호하는 경향입니다.

사용자의 피드백을 조사하여 이러한 편향을 발견하는 것이 중요합니다. 이러한 편향을 이해하면 피드백을 올바르게 해석할 수 있어 잘못된 제품 결정을 피할 수 있습니다.

**퇴화된 피드백 루프(Degenerate feedback loop)**

사용자 피드백은 불완전하다는 점을 기억하세요. 당신이 사용자에게 보여주는 것에 대해서만 피드백을 받을 수 있습니다.

사용자 피드백을 모델의 동작 수정에 사용하는 시스템에서는 퇴화된 피드백 루프가 발생할 수 있습니다. 퇴화된 피드백 루프란 예측이 피드백에 영향을 주고, 이 피드백이 다음 모델 반복에 영향을 줘서 초기 편향을 증폭시키는 현상입니다.

예를 들어, 동영상을 추천하는 시스템을 구축한다고 상상해 보세요. 순위가 높은 동영상이 먼저 표시되므로 더 많은 클릭을 받게 되고, 시스템은 이것이 최고의 선택이라고 생각하게 됩니다. 처음에는 A와 B라는 두 영상 사이에 큰 차이가 없었지만, A가 약간 더 높은 순위를 받아 더 많은 클릭을 받고, 시스템이 이를 계속 강화하게 됩니다. 시간이 지나면서 A의 순위는 치솟고 B는 뒤처지게 됩니다. 이런 피드백 루프 때문에 인기 있는 동영상은 계속 인기를 유지하고, 새로운 동영상은 두각을 나타내기 어렵습니다. 이 문제는 "노출 편향(exposure bias)", "인기도 편향(popularity bias)", 또는 "필터 버블(filter bubbles)"로 알려져 있으며, 잘 연구된 문제입니다.

퇴화된 피드백 루프는 제품의 초점과 사용자 기반을 바꿀 수 있습니다. 처음에 소수의 사용자가 고양이 사진을 좋아한다고 피드백을 줬다고 상상해 보세요. 시스템은 이를 감지하고 더 많은 고양이 사진을 생성합니다. 그러면 고양이를 좋아하는 사람들이 모이게 되고, 이들이 더 많은 긍정 피드백을 주며 시스템은 더 많은 고양이 사진을 생성하게 됩니다. 어느 순간, 애플리케이션은 고양이 천국이 되어버립니다. 여기서는 고양이 사진을 예로 들었지만, 동일한 메커니즘은 인종차별, 성차별, 노골적인 콘텐츠 선호와 같은 다른 편향도 증폭시킬 수 있습니다.

사용자 피드백에 따라 행동하는 것은 대화형 에이전트를, 더 나은 표현이 없다면, 거짓말쟁이로 만들 수 있습니다. 여러 연구들은 모델을 사용자 피드백으로 학습시키면, 가장 정확하거나 유익한 답변이 아니라 사용자가 원하는 것을 말하게 되는 경향이 있다는 것을 보여줬습니다(Stray, 2023). Sharma 등(2023)은 인간 피드백으로 학습된 AI 모델들이 아첨(sycophancy) 성향을 보인다고 밝혔습니다. 이들은 사용자의 견해에 부합하는 응답을 제시할 가능성이 높아졌습니다.

사용자 피드백은 사용자 경험을 개선하는 데 중요하지만, 이를 무분별하게 사용하면 편향을 지속시키고 제품을 망칠 수 있습니다. 피드백을 제품에 통합하기 전에, 이 피드백의 한계와 그 잠재적 영향을 반드시 이해해야 합니다.

---

## **Summary**  

이전 각 장이 AI 엔지니어링의 특정 측면에 초점을 맞췄다면, 이 장은 파운데이션 모델 위에 애플리케이션을 구축하는 전체 프로세스를 살펴보았습니다.

이 장은 두 부분으로 구성되어 있습니다. 첫 번째 부분에서는 AI 애플리케이션의 공통 아키텍처에 대해 논의했습니다. 애플리케이션의 정확한 아키텍처는 다를 수 있지만, 이 고수준 아키텍처는 서로 다른 구성 요소가 어떻게 맞물리는지를 이해하는 틀을 제공합니다. 이 아키텍처를 단계적으로 구축하는 방식을 통해 각 단계에서의 과제와 이를 해결하는 데 사용할 수 있는 기술들을 논의했습니다.

시스템을 모듈화하고 유지 가능하게 만들기 위해 구성 요소를 분리하는 것은 필요하지만, 이 분리는 유동적입니다. 구성 요소가 기능적으로 겹칠 수 있는 방식은 여러 가지가 있습니다. 예를 들어, 가드레일은 추론 서비스나 모델 게이트웨이, 또는 독립 구성 요소로 구현될 수 있습니다.

각 추가 구성 요소는 시스템을 더 강력하고, 더 안전하게 만들거나, 더 빠르게 만들 수 있지만 동시에 시스템의 복잡성을 증가시켜 새로운 실패 모드에 노출시킬 수 있습니다. 복잡한 시스템의 핵심 요소 중 하나는 모니터링과 관측 가능성(observability)입니다.

관측 가능성은 시스템이 어떻게 실패하는지를 이해하고, 이러한 실패에 대한 지표 및 경고를 설계하며, 시스템이 이러한 실패를 감지하고 추적할 수 있도록 설계되었는지를 보장하는 것을 포함합니다. 많은 소프트웨어 엔지니어링 및 전통적인 머신러닝의 관측 가능성 관련 모범 사례와 도구들은 AI 엔지니어링 애플리케이션에도 적용 가능하지만, 파운데이션 모델은 새로운 실패 모드를 도입하기 때문에 추가적인 지표 및 설계 고려사항이 필요합니다.

동시에, 대화형 인터페이스는 새로운 유형의 사용자 피드백을 가능하게 하며, 이를 분석, 제품 개선 및 데이터 플라이휠에 활용할 수 있습니다. 이 장의 두 번째 부분에서는 다양한 형태의 대화형 피드백과 이를 효과적으로 수집할 수 있도록 애플리케이션을 설계하는 방법을 논의했습니다.

전통적으로 사용자 피드백 설계는 제품의 책임으로 간주되었고, 그 결과 엔지니어들이 종종 간과해 왔습니다. 하지만 사용자 피드백은 AI 모델을 지속적으로 개선하기 위한 중요한 데이터 원천이기 때문에, 이제는 더 많은 AI 엔지니어들이 필요한 데이터를 받을 수 있도록 이 프로세스에 적극적으로 참여하고 있습니다. 이는 1장에서 언급했던 것처럼, 전통적인 머신러닝 엔지니어링과 비교할 때 AI 엔지니어링이 제품에 더 가까워지고 있다는 점을 강화합니다.

이는 데이터 플라이휠과 제품 경험이 경쟁력 있는 장점으로 점점 더 중요해지고 있기 때문입니다.

많은 AI 과제는 본질적으로 시스템 문제입니다. 이러한 문제를 해결하기 위해서는 시스템 전체를 한 발 물러서서 바라보는 것이 종종 필요합니다. 단일 문제는 독립적으로 작동하는 여러 구성 요소에 의해 해결될 수 있으며, 또는 여러 구성 요소의 협력이 필요한 경우도 있습니다. 시스템에 대한 철저한 이해는 실제 문제를 해결하고, 새로운 가능성을 열고, 안전성을 보장하는 데 필수적입니다.


---
1. 삼성 직원이 ChatGPT에 삼성의 독점 정보를 입력하여 회사 기밀을 실수로 유출한 사례가 있습니다.
2. 사용자가 모델에게 빈 응답을 반환하도록 요청할 수도 있습니다.
3. 몇몇 초기 독자들은 지연 시간을 줄이기 위해 가드레일을 무시한다는 아이디어에 악몽을 꿨다고 말했습니다.
4. 이 글을 작성하는 시점 기준으로, 대표적인 관측 가능성(Observability) 회사들(Datadog, Splunk, Dynatrace, New Relic)의 총 시가총액은 약 1,000억 달러에 달합니다.
5. 제 책 *Designing Machine Learning Systems* (O'Reilly, 2022)에도 모니터링에 대한 장이 있습니다. 해당 장의 초기 초안은 제 블로그의 "Data Distribution Shifts and Monitoring"에서 확인할 수 있습니다.
6. 이러한 이유로 일부 오케스트레이터 도구는 게이트웨이가 되고 싶어합니다. 실제로 너무나 많은 도구들이 모든 것을 수행하는 엔드-투-엔드 플랫폼이 되기를 원합니다.
7. 오픈 소스 애플리케이션을 상용 애플리케이션 대신 출시할 때의 주요 단점 중 하나는 사용자 피드백을 수집하기 훨씬 어렵다는 점입니다. 사용자는 오픈 소스 애플리케이션을 받아 자체적으로 배포할 수 있고, 개발자는 그 애플리케이션이 어떻게 사용되는지 전혀 알 수 없습니다.
8. AI 애플리케이션에 대한 피드백을 수집할 수 있을 뿐 아니라, AI를 활용해 피드백을 분석할 수도 있습니다.
9. 텍스트-투-스피치(TTS)에 인페인팅(inpainting)이 있으면 좋겠습니다. TTS는 95%는 잘 작동하지만, 나머지 5%는 실망스럽습니다. AI가 이름을 잘못 발음하거나 대화 중간에 멈추지 않는 경우가 있습니다. 전체 오디오를 다시 생성하지 않고, 실수만 편집할 수 있는 앱이 있었으면 좋겠습니다.
10. 제가 참석한 행사에서 이 질문을 하면, 반응은 엇갈립니다. 어떤 사람들은 전체 응답을 보여주는 것이 더 많은 정보를 제공하므로 더 신뢰할 수 있는 피드백을 준다고 생각합니다. 동시에 어떤 사람들은 사용자가 전체 응답을 이미 읽어버리면, 더 나은 쪽을 클릭할 동기가 사라진다고 생각합니다.
11. "Ted Cruz Blames Staffer for 'Liking' Porn Tweet" (Nelson and Everett, POLITICO, 2017년 9월) 및 "Kentucky Senator Whose Twitter Account 'Liked' Obscene Tweets Says He Was Hacked" (Liam Niemeyer, WKU Public Radio, 2023년 3월) 참고.
12. 여기 제시된 옵션들은 단지 옵션을 어떻게 다시 표현할 수 있는지를 보여주기 위한 예시일 뿐이며, 검증된 것은 아닙니다.

---

# Epilogue 

축하합니다! 당신은 이제 150,000개 이상의 단어, 160개의 일러스트레이션, 250개의 각주, 975개의 참고 링크가 포함된 기술서를 완독했습니다.

학습을 위해 시간을 할애할 수 있다는 것은 특권입니다. 이 책을 집필하고 새로운 것들을 배울 수 있었던 기회에 감사드리며, 당신이 이 책에 소중한 학습 시간을 내어주신 것에 진심으로 감사드립니다.

기술 글쓰기에서 가장 어려운 부분은 정답을 찾는 것이 아니라, 올바른 질문을 던지는 것입니다. 이 책을 쓰면서 저는 많은 질문을 하게 되었고, 그 질문들은 저를 흥미롭고 유용한 발견으로 이끌었습니다. 이 책이 당신에게도 흥미로운 질문을 떠올리게 했기를 바랍니다.

이미 파운데이션 모델 위에 수많은 놀라운 애플리케이션들이 구축되어 있습니다. 앞으로 그 수는 기하급수적으로 늘어날 것임에 의심의 여지가 없습니다. 이 책에서 소개한 것과 같은 보다 체계적인 AI 엔지니어링 접근 방식은 개발 과정을 더 쉽게 만들어주어, 더욱 많은 애플리케이션의 등장을 가능하게 할 것입니다. 논의하고 싶은 사용 사례가 있다면,

언제든지 연락 주세요. 흥미로운 문제와 해결책에 대한 이야기를 듣는 것을 좋아합니다. X에서는 @chipro, LinkedIn에서는 /in/chiphuyen, 이메일은 [https://huyenchip.com/communication](https://huyenchip.com/communication) 을 통해 연락하실 수 있습니다.

AI 엔지니어링에 대한 더 많은 자료는 책의 GitHub 저장소 [https://github.com/chiphuyen/aie-book](https://github.com/chiphuyen/aie-book) 를 참고하세요.

AI 엔지니어링에는 많은 도전 과제가 있습니다. 그 모두가 즐거운 것은 아니지만, 그 모두가 성장과 영향력을 위한 기회입니다. 당신이 앞으로 무엇을 만들어갈지 더 알아가고 싶습니다!
